---
## Section 1: Setup & Configuration

In [1]:
# ==============================================================================
# IMPORTS (Standard Libraries Only - No Local Modules)
# ==============================================================================
import os
import glob
import warnings
import copy
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional, Any, Union
warnings.filterwarnings('ignore')

# Core libraries
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

print("All imports successful!")

All imports successful!


### 1.1 Configurations & Results Containers

Task-specific configuration dataclasses and result containers for Task 1 (single-step) and Task 2 (multi-step Seq2Seq).

In [2]:
# ==============================================================================
# TASK 1 CONFIGURATION (Separate, Full Control)
# ==============================================================================

@dataclass
class Task1Config:
    """
    Configuration for Task 1: Single-step LSTM prediction (t+2).
    
    All parameters for Task 1 are consolidated here for full control.
    Instantiate with custom values for easy experimentation:
    
        cfg1 = Task1Config(num_basins=10, epochs=50, hidden_dim=64)
    """
    # === EXPERIMENT SETTINGS ===
    num_basins: int = 5              # 0 = use ALL basins, >0 = limit for quick testing
    use_static: bool = True          # Whether to include static catchment attributes
    
    # === TRAINING HYPERPARAMETERS ===
    epochs: int = 50
    batch_size: int = 256
    learning_rate: float = 0.001
    dropout: float = 0.2
    
    # === EARLY STOPPING ===
    patience: int = 10               # Stop if val_loss doesn't improve for this many epochs
    
    # === MODEL ARCHITECTURE ===
    hidden_dim: int = 64             # Hidden dimension for LSTM
    use_bidirectional: bool = True   # Use bidirectional LSTM
    use_self_attention: bool = True  # Apply multi-head self-attention after LSTM
    num_attention_heads: int = 4     # Number of heads for self-attention
    use_layer_norm: bool = True      # Apply LayerNorm in model
    num_lstm_layers: int = 1         # Number of stacked LSTM layers
    
    # === ADVANCED TRAINING TECHNIQUES (Toggleable) ===
    use_weight_decay: bool = True    # Use AdamW with weight decay (else Adam)
    weight_decay: float = 0.01       # L2 regularization strength
    use_scheduler: bool = True       # Use ReduceLROnPlateau scheduler
    scheduler_patience: int = 5      # Scheduler patience before reducing LR
    scheduler_factor: float = 0.5    # Factor to reduce LR by
    use_gradient_clipping: bool = True  # Apply gradient clipping
    gradient_clip_value: float = 1.0    # Max gradient norm
    
    # === VISUALIZATION ===
    show_plots: bool = True          # Set False to suppress plots during batch experiments
    
    # === DEVICE ===
    device_type: str = 'auto'        # 'auto', 'cuda', 'cpu'
    
    # === PATHS ===
    base_dir: str = './basin_dataset_public'
    results_dir: str = './results/task1'
    
    # === SEQUENCE PARAMETERS ===
    seq_length: int = 60             # Lookback window (days)
    predict_horizon: int = 2         # Predict t + k
    
    # === DATA SPLIT (Hydrological Years) ===
    train_start: str = '1980-10-01'
    train_end: str = '1995-09-30'
    val_start: str = '1995-10-01'
    val_end: str = '2000-09-30'
    test_start: str = '2000-10-01'
    test_end: str = '2010-09-30'
    
    # === FEATURE SELECTION ===
    dynamic_features: List[str] = field(default_factory=lambda: [
        'PRCP', 'SRAD', 'Tmax', 'Tmin', 'Vp',   # Original Forcing
        'PRCP_roll3', 'PRCP_roll7',             # Rolling Stats
        'Q_lag1', 'Q_lag2', 'Q_lag3'            # Lag Features
    ])
    
    static_features: List[str] = field(default_factory=lambda: [
        'area_gages2',  # Catchment Area (Will be Log transformed)
        'elev_mean',    # Mean Elevation
        'slope_mean',   # Topography
        'sand_frac',    # Soil Type
        'clay_frac',    # Soil Type
        'frac_forest',  # Vegetation
        'lai_max',      # Leaf Area Index
        'p_mean',       # Long-term climate
        'aridity'       # Climate Index
    ])
    
    target: str = 'Q_cms'
    
    # === CONSTANTS ===
    cfs_to_cms: float = 0.0283168    # Cubic feet/sec to cubic meters/sec
    epsilon: float = 1e-6            # Small constant for numerical stability
    
    # === DERIVED PATHS (computed in __post_init__) ===
    forcing_dir: str = field(init=False)
    flow_dir: str = field(init=False)
    meta_dir: str = field(init=False)
    bad_basins_file: str = field(init=False)
    device: torch.device = field(init=False)
    
    def __post_init__(self):
        """Compute derived attributes after initialization."""
        self.forcing_dir = os.path.join(self.base_dir, 'basin_mean_forcing', 'nldas')
        self.flow_dir = os.path.join(self.base_dir, 'usgs_streamflow')
        self.meta_dir = self.base_dir
        self.bad_basins_file = os.path.join(self.base_dir, 'basin_size_errors_10_percent.txt')
        
        if self.device_type == 'auto':
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(self.device_type)
    
    def print_config(self):
        """Pretty print the configuration."""
        print("=" * 60)
        print("TASK 1 CONFIGURATION")
        print("=" * 60)
        print(f"\n--- Experiment Settings ---")
        print(f"Number of Basins: {self.num_basins} {'(ALL)' if self.num_basins == 0 else '(limited)'}")
        print(f"Use Static Features: {self.use_static}")
        
        print(f"\n--- Training Hyperparameters ---")
        print(f"Epochs: {self.epochs}, Batch Size: {self.batch_size}")
        print(f"Learning Rate: {self.learning_rate}, Dropout: {self.dropout}")
        print(f"Early Stopping Patience: {self.patience}")
        
        print(f"\n--- Model Architecture ---")
        print(f"LSTM Layers: {self.num_lstm_layers}")
        print(f"Bidirectional LSTM: {self.use_bidirectional}")
        print(f"Self-Attention: {self.use_self_attention}" + 
              (f" (Heads: {self.num_attention_heads})" if self.use_self_attention else ""))
        print(f"Layer Normalization: {self.use_layer_norm}")
        
        print(f"\n--- Advanced Training Techniques ---")
        print(f"Weight Decay (AdamW): {self.use_weight_decay}" + 
              (f" (λ={self.weight_decay})" if self.use_weight_decay else ""))
        print(f"LR Scheduler: {self.use_scheduler}" + 
              (f" (patience={self.scheduler_patience}, factor={self.scheduler_factor})" if self.use_scheduler else ""))
        print(f"Gradient Clipping: {self.use_gradient_clipping}" + 
              (f" (max_norm={self.gradient_clip_value})" if self.use_gradient_clipping else ""))
        
        print(f"\n--- Device ---")
        print(f"Device: {self.device}")
        if self.device.type == 'cuda':
            print(f"GPU: {torch.cuda.get_device_name(0)}")
        
        print(f"\n--- Sequence Parameters ---")
        print(f"Sequence Length: {self.seq_length} days")
        print(f"Prediction Horizon: t+{self.predict_horizon}")
        print("=" * 60)


# ==============================================================================
# TASK 2 CONFIGURATION (Separate, Full Control)
# ==============================================================================

@dataclass
class Task2Config:
    """
    Configuration for Task 2: Multi-step Seq2Seq prediction (t+1 to t+5).
    
    All parameters for Task 2 are consolidated here for full control.
    Instantiate with custom values for easy experimentation:
    
        cfg2 = Task2Config(num_basins=10, epochs=50, hidden_dim=128)
    """
    # === EXPERIMENT SETTINGS ===
    num_basins: int = 5              # 0 = use ALL basins, >0 = limit for quick testing
    use_static: bool = True          # Whether to include static catchment attributes
    
    # === TRAINING HYPERPARAMETERS ===
    epochs: int = 50
    batch_size: int = 256
    learning_rate: float = 0.001
    dropout: float = 0.2
    
    # === EARLY STOPPING ===
    patience: int = 10               # Stop if val_loss doesn't improve for this many epochs
    
    # === MODEL ARCHITECTURE ===
    hidden_dim: int = 128            # Hidden dimension for Seq2Seq (larger for complexity)
    use_bidirectional_encoder: bool = True   # Use bidirectional encoder
    use_self_attention: bool = True          # Apply self-attention in encoder
    num_attention_heads: int = 4             # Number of heads for self-attention
    use_layer_norm: bool = True              # Apply LayerNorm in model
    num_encoder_layers: int = 1      # Number of stacked LSTM layers in encoder
    num_decoder_layers: int = 1      # Number of stacked LSTMCell layers in decoder
    
    # === TEACHER FORCING ===
    teacher_forcing_ratio: float = 0.5  # Initial teacher forcing probability
    tf_decay: bool = True                # Whether to decay TF ratio over epochs
    
    # === ADVANCED TRAINING TECHNIQUES (Toggleable) ===
    use_weight_decay: bool = True    # Use AdamW with weight decay (else Adam)
    weight_decay: float = 0.01       # L2 regularization strength
    use_scheduler: bool = True       # Use ReduceLROnPlateau scheduler
    scheduler_patience: int = 5      # Scheduler patience before reducing LR
    scheduler_factor: float = 0.5    # Factor to reduce LR by
    use_gradient_clipping: bool = True  # Apply gradient clipping
    gradient_clip_value: float = 1.0    # Max gradient norm
    
    # === VISUALIZATION ===
    show_plots: bool = True          # Set False to suppress plots during batch experiments
    
    # === DEVICE ===
    device_type: str = 'auto'        # 'auto', 'cuda', 'cpu'
    
    # === PATHS ===
    base_dir: str = './basin_dataset_public'
    results_dir: str = './results/task2'
    
    # === SEQUENCE PARAMETERS ===
    seq_length: int = 60             # Lookback window (days)
    predict_steps: int = 5           # Predict sequence t+1...t+5
    
    # === DATA SPLIT (Hydrological Years) ===
    train_start: str = '1980-10-01'
    train_end: str = '1995-09-30'
    val_start: str = '1995-10-01'
    val_end: str = '2000-09-30'
    test_start: str = '2000-10-01'
    test_end: str = '2010-09-30'
    
    # === FEATURE SELECTION ===
    dynamic_features: List[str] = field(default_factory=lambda: [
        'PRCP', 'SRAD', 'Tmax', 'Tmin', 'Vp',   # Original Forcing
        'PRCP_roll3', 'PRCP_roll7',             # Rolling Stats
        'Q_lag1', 'Q_lag2', 'Q_lag3'            # Lag Features
    ])
    
    static_features: List[str] = field(default_factory=lambda: [
        'area_gages2',  # Catchment Area (Will be Log transformed)
        'elev_mean',    # Mean Elevation
        'slope_mean',   # Topography
        'sand_frac',    # Soil Type
        'clay_frac',    # Soil Type
        'frac_forest',  # Vegetation
        'lai_max',      # Leaf Area Index
        'p_mean',       # Long-term climate
        'aridity'       # Climate Index
    ])
    
    # Future forcing features (for Seq2Seq decoder)
    future_forcing_features: List[str] = field(default_factory=lambda: [
        'PRCP', 'SRAD', 'Tmax', 'Tmin', 'Vp'
    ])
    target: str = 'Q_cms'
    target: str = 'Q_cms'
    
    # === CONSTANTS ===
    cfs_to_cms: float = 0.0283168    # Cubic feet/sec to cubic meters/sec
    epsilon: float = 1e-6            # Small constant for numerical stability
    
    # === DERIVED PATHS (computed in __post_init__) ===
    forcing_dir: str = field(init=False)
    flow_dir: str = field(init=False)
    meta_dir: str = field(init=False)
    bad_basins_file: str = field(init=False)
    device: torch.device = field(init=False)
    
    def __post_init__(self):
        """Compute derived attributes after initialization."""
        self.forcing_dir = os.path.join(self.base_dir, 'basin_mean_forcing', 'nldas')
        self.flow_dir = os.path.join(self.base_dir, 'usgs_streamflow')
        self.meta_dir = self.base_dir
        self.bad_basins_file = os.path.join(self.base_dir, 'basin_size_errors_10_percent.txt')
        
        if self.device_type == 'auto':
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(self.device_type)
    
    def print_config(self):
        """Pretty print the configuration."""
        print("TASK 2 CONFIGURATION")
        print("TASK 2 CONFIGURATION")
        print("=" * 60)
        print(f"\n--- Experiment Settings ---")
        print(f"Number of Basins: {self.num_basins} {'(ALL)' if self.num_basins == 0 else '(limited)'}")
        print(f"Use Static Features: {self.use_static}")
        
        print(f"\n--- Training Hyperparameters ---")
        print(f"Epochs: {self.epochs}, Batch Size: {self.batch_size}")
        print(f"Learning Rate: {self.learning_rate}, Dropout: {self.dropout}")
        print(f"Early Stopping Patience: {self.patience}")
        
        print(f"Encoder Layers: {self.num_encoder_layers}, Decoder Layers: {self.num_decoder_layers}")
        print(f"Bidirectional Encoder: {self.use_bidirectional_encoder}")
        print(f"Encoder Self-Attention: {self.use_self_attention}" + 
              (f" (Heads: {self.num_attention_heads})" if self.use_self_attention else ""))
        print(f"Layer Normalization: {self.use_layer_norm}")
        print(f"Teacher Forcing: {self.teacher_forcing_ratio} (Decay: {self.tf_decay})")
        
        print(f"\n--- Advanced Training Techniques ---")
        print(f"Weight Decay (AdamW): {self.use_weight_decay}" + 
              (f" (λ={self.weight_decay})" if self.use_weight_decay else ""))
        print(f"LR Scheduler: {self.use_scheduler}" + 
              (f" (patience={self.scheduler_patience}, factor={self.scheduler_factor})" if self.use_scheduler else ""))
        print(f"Gradient Clipping: {self.use_gradient_clipping}" + 
              (f" (max_norm={self.gradient_clip_value})" if self.use_gradient_clipping else ""))
        
        print(f"\n--- Device ---")
        print(f"Device: {self.device}")
        if self.device.type == 'cuda':
            print(f"GPU: {torch.cuda.get_device_name(0)}")
        
        print(f"\n--- Sequence Parameters ---")
        print(f"Sequence Length: {self.seq_length} days")
        print(f"Prediction Steps: {self.predict_steps} (t+1 to t+{self.predict_steps})")
        print("=" * 60)
        print("=" * 60)


print("Task1Config and Task2Config defined.")

print("Task1Config and Task2Config defined.")
print("Task1Config and Task2Config defined.")
print("Task1Config and Task2Config defined.")

Task1Config and Task2Config defined.
Task1Config and Task2Config defined.
Task1Config and Task2Config defined.
Task1Config and Task2Config defined.


#### (Continued) Results Containers

Dataclasses for storing pipeline outputs.

In [3]:
# ==============================================================================
# TASK 1 RESULTS CONTAINER
# ==============================================================================

@dataclass
class Task1Results:
    """
    Container for Task 1 pipeline outputs.
    
    Stores the trained model, training history, evaluation metrics,
    predictions, and metadata from the Task 1 pipeline.
    """
    model: nn.Module                         # Trained LSTMWithAttention model
    history: Dict[str, List[float]]          # {'train_loss': [...], 'val_loss': [...]}
    metrics: Dict[str, float]                # {'MSE', 'RMSE', 'MAE', 'NSE', 'R2'}
    predictions: np.ndarray                  # (N,) predictions on test set
    targets: np.ndarray                      # (N,) ground truth on test set
    basin_ids: List[str]                     # List of basin IDs used
    preprocessor: Any                        # CamelsPreprocessor for inverse transform
    
    def summary(self):
        """Print a summary of Task 1 results."""
        print("\n" + "=" * 60)
        print("TASK 1 RESULTS SUMMARY")
        print("=" * 60)
        print(f"Basins processed: {len(self.basin_ids)}")
        print(f"Test samples: {len(self.predictions)}")
        print(f"\nTest Metrics:")
        for k, v in self.metrics.items():
            print(f"  {k}: {v:.4f}")
        print(f"\nTraining epochs: {len(self.history['train_loss'])}")
        print(f"Best val loss: {min(self.history['val_loss']):.6f}")
        print("=" * 60)


# ==============================================================================
# TASK 2 RESULTS CONTAINER
# ==============================================================================

@dataclass
class Task2Results:
    """
    Container for Task 2 pipeline outputs.
    
    Stores the trained model, training history, evaluation metrics (overall and per-step),
    predictions, and metadata from the Task 2 pipeline.
    """
    model: nn.Module                              # Trained LSTM_Seq2Seq model
    history: Dict[str, List[float]]               # {'train_loss': [...], 'val_loss': [...]}
    metrics: Dict[str, float]                     # Averaged metrics across all forecast steps
    metrics_per_step: Dict[int, Dict[str, float]] # {1: {...}, 2: {...}, ...} per-horizon metrics
    predictions: np.ndarray                       # (N, 5) predictions on test set
    targets: np.ndarray                           # (N, 5) ground truth on test set
    basin_ids: List[str]                          # List of basin IDs used
    preprocessor: Any                             # CamelsPreprocessor for inverse transform
    
    def summary(self):
        """Print a summary of Task 2 results."""
        print("\n" + "=" * 60)
        print("TASK 2 RESULTS SUMMARY")
        print("=" * 60)
        print(f"Basins processed: {len(self.basin_ids)}")
        print(f"Test samples: {len(self.predictions)}")
        print(f"\nTest Metrics (averaged across {len(self.metrics_per_step)} steps):")
        for k, v in self.metrics.items():
            print(f"  {k}: {v:.4f}")
        print(f"\nPer-Step NSE:")
        for step, step_metrics in self.metrics_per_step.items():
            print(f"  t+{step}: NSE={step_metrics['NSE']:.4f}, RMSE={step_metrics['RMSE']:.4f}")
        print(f"\nTraining epochs: {len(self.history['train_loss'])}")
        print(f"Best val loss: {min(self.history['val_loss']):.6f}")
        print("=" * 60)


print("Task1Results and Task2Results containers defined.")

Task1Results and Task2Results containers defined.


### 1.2 Data Pipeline (Loader, Engineering, Preprocessing)

Classes for loading CAMELS data, engineering features, and preprocessing.

In [4]:
# ==============================================================================
# DATA LOADER CLASS
# ==============================================================================

class CamelsLoader:
    """
    Data loader for CAMELS dataset.
    Handles loading streamflow, forcing data, and static attributes.
    """
    def __init__(self, cfg: Union[Task1Config, Task2Config]):
        self.cfg = cfg

    def load_bad_basins(self) -> List[str]:
        """Returns a list of basin IDs to exclude due to data quality issues."""
        if not os.path.exists(self.cfg.bad_basins_file):
            return []
        
        bad_ids = []
        try:
            with open(self.cfg.bad_basins_file, 'r') as f:
                next(f)  # Skip header
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        bad_ids.append(parts[1])
        except Exception as e:
            print(f"Failed to parse bad basins file: {e}")
        return bad_ids

    def get_basin_list(self) -> pd.DataFrame:
        """Scans directories and returns valid basin list (excluding bad basins)."""
        bad_basins = self.load_bad_basins()
        
        search_path = os.path.join(self.cfg.flow_dir, '**', '*_streamflow_qc.txt')
        files = glob.glob(search_path, recursive=True)
        
        basins = []
        for f in files:
            parts = f.split(os.sep)
            region = parts[-2]
            gauge_id = parts[-1].split('_')[0]
            
            if gauge_id not in bad_basins:
                basins.append({'gauge_id': gauge_id, 'region': region})
                
        return pd.DataFrame(basins)

    def load_dynamic_data(self, gauge_id: str, region: str) -> Optional[pd.DataFrame]:
        """
        Loads Streamflow + Forcing data for a single basin.
        
        Args:
            gauge_id: USGS gauge identifier
            region: HUC region code
            
        Returns:
            DataFrame with columns: Q_cms, PRCP, SRAD, Tmax, Tmin, Vp
            Index: DatetimeIndex
        """
        # 1. Load Streamflow
        flow_path = os.path.join(self.cfg.flow_dir, region, f'{gauge_id}_streamflow_qc.txt')
        try:
            df_flow = pd.read_csv(flow_path, sep=r'\s+', header=None,
                                  names=['gauge_id', 'Year', 'Month', 'Day', 'Q_cfs', 'QC'])
        except:
            return None

        df_flow['Date'] = pd.to_datetime(df_flow[['Year', 'Month', 'Day']])
        df_flow.set_index('Date', inplace=True)
        # Convert to CMS and mark missing values
        df_flow['Q_cms'] = df_flow['Q_cfs'].replace(-999, np.nan) * self.cfg.cfs_to_cms

        # 2. Load Forcing (NLDAS)
        forcing_path = os.path.join(self.cfg.forcing_dir, region, f'{gauge_id}_lump_nldas_forcing_leap.txt')
        if not os.path.exists(forcing_path):
            return None

        try:
            df_force = pd.read_csv(forcing_path, sep=r'\s+', skiprows=3)
        except:
            return None

        # Standardize column names
        col_map = {
            'Mnth': 'Month', 'month': 'Month', 'mo': 'Month',
            'year': 'Year', 'yr': 'Year',
            'day': 'Day', 'dy': 'Day',
            'prcp(mm/day)': 'PRCP', 'srad(w/m2)': 'SRAD', 
            'tmax(c)': 'Tmax', 'tmin(c)': 'Tmin', 'vp(pa)': 'Vp'
        }
        
        new_cols = {}
        for c in df_force.columns:
            clean = c.strip()
            if clean.lower() in col_map:
                new_cols[c] = col_map[clean.lower()]
            elif clean in col_map:
                new_cols[c] = col_map[clean]
        
        df_force.rename(columns=new_cols, inplace=True)
        
        # Create Date Index
        try:
            df_force['Date'] = pd.to_datetime(df_force[['Year', 'Month', 'Day']])
            df_force.set_index('Date', inplace=True)
        except KeyError:
            return None

        # 3. Merge (inner join to ensure alignment)
        base_forcing = ['PRCP', 'SRAD', 'Tmax', 'Tmin', 'Vp']
        cols_to_use = [c for c in base_forcing if c in df_force.columns]
        df_merged = df_flow[['Q_cms']].join(df_force[cols_to_use], how='inner')
        
        return df_merged

    def load_static_attributes(self, basins_list: List[str] = None) -> Optional[pd.DataFrame]:
        """
        Loads all static attribute files, merges them, and filters for configured features.
        
        Args:
            basins_list: List of gauge_ids to include (filters result)
            
        Returns:
            DataFrame with static features, indexed by gauge_id
        """
        files = ['camels_topo.txt', 'camels_soil.txt', 'camels_clim.txt', 
                 'camels_vege.txt', 'camels_geol.txt']
        
        dfs = []
        for filename in files:
            path = os.path.join(self.cfg.meta_dir, filename)
            if os.path.exists(path):
                try:
                    df = pd.read_csv(path, sep=';')
                    df.columns = [c.strip() for c in df.columns]
                    if 'gauge_id' in df.columns:
                        df['gauge_id'] = df['gauge_id'].astype(str).str.zfill(8)
                        df.set_index('gauge_id', inplace=True)
                        dfs.append(df)
                except:
                    pass
        
        if not dfs:
            return None

        # Merge all static files
        df_static = pd.concat(dfs, axis=1)
        # Remove duplicate columns
        df_static = df_static.loc[:, ~df_static.columns.duplicated()]

        # Filter for configured features
        available_feats = [f for f in self.cfg.static_features if f in df_static.columns]
        df_final = df_static[available_feats]

        if basins_list is not None:
            df_final = df_final.reindex(basins_list)
            
        return df_final

print("CamelsLoader class defined.")

CamelsLoader class defined.


#### (Continued) Feature Engineering

Rolling statistics and lag feature computation.

In [5]:
# ==============================================================================
# FEATURE ENGINEERING CLASS
# ==============================================================================

class FeatureEngineer:
    """
    Creates derived features from raw hydrometeorological data.
    
    Features created:
    - PRCP_roll3: 3-day rolling mean precipitation (short-term wetness)
    - PRCP_roll7: 7-day rolling mean precipitation (medium-term saturation)
    - Q_lag1, Q_lag2, Q_lag3: Lagged streamflow values
    """
    def __init__(self, cfg: Union[Task1Config, Task2Config]):
        self.cfg = cfg

    def add_rolling_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculates rolling statistics for Precipitation.
        Represents accumulated soil moisture / wetness.
        """
        if 'PRCP' in df.columns:
            # 3-Day Rolling Mean (Short-term wetness)
            df['PRCP_roll3'] = df['PRCP'].rolling(window=3, min_periods=1).mean()
            # 7-Day Rolling Mean (Medium-term saturation)
            df['PRCP_roll7'] = df['PRCP'].rolling(window=7, min_periods=1).mean()
        return df

    def add_lag_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Adds explicit lag features for Streamflow.
        Gives the model explicit access to past flow values.
        
        Note: Shifting creates NaNs at the top of the dataframe.
        These will be handled by the preprocessor's handle_missing_data().
        """
        target = self.cfg.target  # 'Q_cms'
        
        if target in df.columns:
            df['Q_lag1'] = df[target].shift(1)  # Flow yesterday (t-1)
            df['Q_lag2'] = df[target].shift(2)  # Flow 2 days ago (t-2)
            df['Q_lag3'] = df[target].shift(3)  # Flow 3 days ago (t-3)
        return df

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Applies all feature engineering steps.
        Should be run BEFORE missing data interpolation.
        """
        df = self.add_rolling_features(df)
        df = self.add_lag_features(df)
        return df

print("FeatureEngineer class defined.")

FeatureEngineer class defined.


#### (Continued) Preprocessor

Target transformation, scaling, and data splitting.

In [6]:
# ==============================================================================
# PREPROCESSOR CLASS
# ==============================================================================

class CamelsPreprocessor:
    """
    Preprocessor for CAMELS data.
    
    Handles:
    - Physical outlier cleaning (negative rain/flow → 0, unrealistic temps → NaN)
    - Missing data interpolation (with gap limits)
    - Cyclical date encoding (sin/cos of day-of-year)
    - Normalization (global dynamic stats, basin-specific target stats)
    - Sequence generation for LSTM input
    """
    def __init__(self, cfg: Union[Task1Config, Task2Config]):
        self.cfg = cfg
        self.scalers = {}
        self.basin_scalers = {}
        
        # Physical Constraints for data cleaning
        self.PHYSICAL_LIMITS = {
            'PRCP': {'min': 0.0, 'max': None},
            'Q_cms': {'min': 0.0, 'max': None},
            'Tmax': {'min': -60.0, 'max': 60.0},
            'Tmin': {'min': -60.0, 'max': 60.0}
        }
        self.MAX_INTERPOLATE_GAP = 5  # Max consecutive days to interpolate

    def add_date_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Adds cyclical encoding of day-of-year."""
        day_of_year = df.index.dayofyear
        df['sin_doy'] = np.sin(2 * np.pi * day_of_year / 365.0)
        df['cos_doy'] = np.cos(2 * np.pi * day_of_year / 365.0)
        return df

    def clean_physical_outliers(self, df: pd.DataFrame) -> pd.DataFrame:
        """Enforces physical constraints on the data."""
        # Negative Rain/Flow → 0
        for col in ['PRCP', self.cfg.target]:
            if col in df.columns:
                mask = df[col] < 0
                if mask.any():
                    df.loc[mask, col] = 0.0
        
        # Unrealistic Temp → NaN
        for col in ['Tmax', 'Tmin']:
            if col in df.columns:
                limits = self.PHYSICAL_LIMITS[col]
                mask = (df[col] < limits['min']) | (df[col] > limits['max'])
                if mask.any():
                    df.loc[mask, col] = np.nan
        return df

    def handle_missing_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Interpolates missing values with gap limit."""
        cols_to_fix = [self.cfg.target] + self.cfg.dynamic_features
        cols_to_fix = [c for c in cols_to_fix if c in df.columns]

        for col in cols_to_fix:
            # Linear interpolate short gaps only
            df[col] = df[col].interpolate(method='linear', limit=self.MAX_INTERPOLATE_GAP, limit_direction='both')
            # Handle edges with forward/backward fill
            df[col] = df[col].ffill().bfill()
        return df

    def fit(self, dynamic_data_dict: Dict[str, pd.DataFrame], static_df: pd.DataFrame = None):
        """
        Computes normalization statistics from TRAINING data only.
        
        Args:
            dynamic_data_dict: {gauge_id: DataFrame} of processed dynamic data
            static_df: DataFrame of static features (optional)
        """
        print("Computing global statistics from training data...")
        
        # 1. Dynamic Feature Stats (global across all basins)
        dyn_vals = []
        for gid, df in dynamic_data_dict.items():
            train_slice = df.loc[self.cfg.train_start:self.cfg.train_end]
            if not train_slice.empty:
                # Only use configured dynamic features
                available_cols = [c for c in self.cfg.dynamic_features if c in train_slice.columns]
                valid_rows = train_slice[available_cols].dropna()
                if not valid_rows.empty:
                    dyn_vals.append(valid_rows.values)
        
        if dyn_vals:
            all_dyn = np.vstack(dyn_vals)
            self.scalers['dynamic_mean'] = np.mean(all_dyn, axis=0)
            self.scalers['dynamic_std'] = np.std(all_dyn, axis=0) + self.cfg.epsilon
        else:
            self.scalers['dynamic_mean'] = 0
            self.scalers['dynamic_std'] = 1

        # 2. Static Feature Stats (with log-transform for area)
        if static_df is not None:
            static_df_copy = static_df.copy()
            if 'area_gages2' in static_df_copy.columns:
                static_df_copy['area_gages2'] = np.log10(np.maximum(static_df_copy['area_gages2'], 1e-3))
            self.scalers['static_mean'] = static_df_copy.mean().values
            self.scalers['static_std'] = static_df_copy.std().values + self.cfg.epsilon
        else:
            print("-> Skipping Static Stats (Static Data not provided)")

        # 3. Basin-specific Target Stats
        print("Computing basin-specific target statistics...")
        for gid, df in dynamic_data_dict.items():
            train_slice = df.loc[self.cfg.train_start:self.cfg.train_end]
            clean_target = train_slice[self.cfg.target].dropna()
            
            if not clean_target.empty:
                self.basin_scalers[gid] = {'mean': clean_target.mean(), 'std': clean_target.std() + self.cfg.epsilon}
            else:
                self.basin_scalers[gid] = {'mean': 0, 'std': 1}

    def transform(self, df_dynamic: pd.DataFrame, df_static: pd.DataFrame = None, 
                  gauge_id: str = None) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """
        Normalizes data and creates feature matrix.
        
        Returns:
            data_matrix: (Time, Features) normalized array
            static_norm: Normalized static vector or None
        """
        # 1. Normalize Dynamic Features
        dyn_cols = [c for c in self.cfg.dynamic_features if c in df_dynamic.columns]
        X_dyn = df_dynamic[dyn_cols].values
        X_dyn_norm = (X_dyn - self.scalers['dynamic_mean']) / self.scalers['dynamic_std']
        
        # 2. Normalize Target (basin-specific)
        target = df_dynamic[self.cfg.target].values
        b_stats = self.basin_scalers.get(gauge_id, {'mean': 0, 'std': 1})
        y_norm = (target - b_stats['mean']) / b_stats['std']
        
        # 3. Normalize Static Features (if available)
        X_stat_norm = None
        if df_static is not None and gauge_id in df_static.index:
            static_vals = df_static.loc[gauge_id].values.copy()
            if 'area_gages2' in df_static.columns:
                area_idx = df_static.columns.get_loc('area_gages2')
                static_vals[area_idx] = np.log10(np.maximum(static_vals[area_idx], 1e-3))
            X_stat_norm = (static_vals - self.scalers['static_mean']) / self.scalers['static_std']
        
        # 4. Get Date Features
        date_feats = df_dynamic[['sin_doy', 'cos_doy']].values
        
        # Combine: [Dynamic_Norm, Date, Target_Norm]
        data_matrix = np.column_stack([X_dyn_norm, date_feats, y_norm])
        
        return data_matrix, X_stat_norm

    def create_sequences(self, data_matrix: np.ndarray, static_vec: np.ndarray = None, 
                         mode: str = 'task1') -> Tuple[np.ndarray, np.ndarray]:
        """
        Creates sliding window sequences for LSTM.
        
        Args:
            data_matrix: (Time, Features) from transform()
            static_vec: Normalized static features or None
            mode: 'task1' (single-step) or 'task2' (multi-step)
            
        Returns:
            X_seq: (N_samples, Seq_Len, Features)
            y_seq: (N_samples,) for task1 or (N_samples, 5) for task2
        """
        X_seq, y_seq = [], []
        seq_len = self.cfg.seq_length
        total_samples = len(data_matrix)
        
        use_static = (static_vec is not None)
        if use_static:
            static_repeated = np.tile(static_vec, (seq_len, 1))

        if mode == 'task1':
            horizon = self.cfg.predict_horizon
            for t in range(seq_len, total_samples - horizon + 1):
                window_data = data_matrix[t-seq_len:t, :]
                target_val = data_matrix[t + horizon - 1, -1]
                
                # Skip sequences with NaN
                if np.isnan(window_data).any() or np.isnan(target_val):
                    continue
                
                if use_static:
                    full_X = np.hstack([window_data, static_repeated])
                else:
                    full_X = window_data
                
                X_seq.append(full_X)
                y_seq.append(target_val)

        elif mode == 'task2':
            steps = self.cfg.predict_steps
            for t in range(seq_len, total_samples - steps + 1):
                window_data = data_matrix[t-seq_len:t, :]
                target_seq = data_matrix[t:t+steps, -1]
                
                if np.isnan(window_data).any() or np.isnan(target_seq).any():
                    continue
                
                if use_static:
                    full_X = np.hstack([window_data, static_repeated])
                else:
                    full_X = window_data
                
                X_seq.append(full_X)
                y_seq.append(target_seq)
                
        return np.array(X_seq), np.array(y_seq)

print("CamelsPreprocessor class defined.")

CamelsPreprocessor class defined.


### 1.3 Neural Network Modules

Attention mechanisms and model architectures for Task 1 (LSTM) and Task 2 (Seq2Seq).

In [7]:
# ==============================================================================
# MULTI-HEAD SELF-ATTENTION MODULE
# ==============================================================================

class MultiHeadSelfAttention(nn.Module):
    """
    Multi-Head Self-Attention for sequence modeling.
    
    Computes scaled dot-product attention over the input sequence,
    allowing the model to attend to information from different positions.
    
    Architecture:
        Input: (Batch, Seq_Len, Hidden)
        Split into num_heads attention heads
        Each head: Q, K, V projections → Scaled Dot-Product → Concat
        Output: (Batch, Seq_Len, Hidden)
    
    Args:
        hidden_dim: Dimension of input and output features
        num_heads: Number of attention heads (must divide hidden_dim)
        dropout: Dropout rate on attention weights
        use_layer_norm: Whether to apply LayerNorm after attention
    """
    def __init__(self, hidden_dim: int, num_heads: int = 4, 
                 dropout: float = 0.1, use_layer_norm: bool = True):
        super(MultiHeadSelfAttention, self).__init__()
        
        assert hidden_dim % num_heads == 0, \
            f"hidden_dim ({hidden_dim}) must be divisible by num_heads ({num_heads})"
        
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        self.scale = self.head_dim ** -0.5  # 1/sqrt(d_k) for scaling
        
        # Linear projections for Q, K, V
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)
        
        # Output projection
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)
        
        # Regularization
        self.dropout = nn.Dropout(dropout)
        
        # Optional Layer Normalization (pre-norm style)
        self.use_layer_norm = use_layer_norm
        if use_layer_norm:
            self.layer_norm = nn.LayerNorm(hidden_dim)
    
    def forward(self, x: torch.Tensor, 
                return_weights: bool = False) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        Forward pass with optional attention weight return.
        
        Args:
            x: Input tensor of shape (Batch, Seq_Len, Hidden)
            return_weights: If True, also return attention weights
            
        Returns:
            out: Output tensor of shape (Batch, Seq_Len, Hidden)
            weights: (optional) Attention weights (Batch, Heads, Seq, Seq)
        """
        batch_size, seq_len, _ = x.size()
        
        # Apply layer norm before attention (pre-norm)
        residual = x
        if self.use_layer_norm:
            x = self.layer_norm(x)
        
        # Project to Q, K, V: (Batch, Seq, Hidden)
        Q = self.q_proj(x)
        K = self.k_proj(x)
        V = self.v_proj(x)
        
        # Reshape for multi-head: (Batch, Heads, Seq, Head_Dim)
        Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled Dot-Product Attention
        # scores: (Batch, Heads, Seq, Seq)
        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        
        # Softmax and dropout
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # Apply attention to values: (Batch, Heads, Seq, Head_Dim)
        attn_output = torch.matmul(attn_weights, V)
        
        # Concatenate heads: (Batch, Seq, Hidden)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.hidden_dim)
        
        # Output projection
        out = self.out_proj(attn_output)
        out = self.dropout(out)
        
        # Residual connection
        out = out + residual
        
        if return_weights:
            return out, attn_weights
        return out

print("MultiHeadSelfAttention module defined.")

MultiHeadSelfAttention module defined.


#### (Continued) LSTMWithAttention Model (Task 1)

Stacked LSTM with optional bidirectional, self-attention, and layer normalization.

In [8]:
# ==============================================================================
# ENHANCED LSTM MODEL WITH ATTENTION (Task 1: Single-Step Prediction)
# ==============================================================================

class LSTMWithAttention(nn.Module):
    """
    Enhanced LSTM for Single-Step Prediction.
    
    Architecture Options:
        - Bidirectional LSTM: Processes sequence forward and backward
        - Stacked LSTM Layers: Multiple LSTM layers for deeper representations
        - Multi-Head Self-Attention: Applied after LSTM for global context
        - Layer Normalization: Stabilizes training
    
    Input: (Batch, Seq_Len, Features)
    Output: Single scalar (normalized flow at t+k)
    
    Args:
        input_dim: Number of input features per timestep
        hidden_dim: Number of LSTM hidden units
        dropout: Dropout rate
        num_layers: Number of stacked LSTM layers
        use_bidirectional: Use bidirectional LSTM
        use_self_attention: Apply self-attention after LSTM
        num_attention_heads: Number of heads for multi-head attention
        use_layer_norm: Apply layer normalization
    """
    def __init__(self, input_dim: int, hidden_dim: int, dropout: float = 0.2,
                 num_layers: int = 1, use_bidirectional: bool = True, 
                 use_self_attention: bool = True,
                 num_attention_heads: int = 4, use_layer_norm: bool = True):
        super(LSTMWithAttention, self).__init__()
        
        self.use_bidirectional = use_bidirectional
        self.use_self_attention = use_self_attention
        self.use_layer_norm = use_layer_norm
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Bidirectional doubles the effective hidden dimension
        self.lstm_output_dim = hidden_dim * 2 if use_bidirectional else hidden_dim
        
        # --- LSTM Layer(s) ---
        # Dropout is applied between LSTM layers (not after the last layer)
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=use_bidirectional,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # --- Optional Layer Norm after LSTM ---
        if use_layer_norm:
            self.lstm_layer_norm = nn.LayerNorm(self.lstm_output_dim)
        
        # --- Optional Self-Attention ---
        if use_self_attention:
            # Ensure hidden dim is divisible by num_heads
            if self.lstm_output_dim % num_attention_heads != 0:
                # Adjust num_heads to a valid divisor
                for n in range(num_attention_heads, 0, -1):
                    if self.lstm_output_dim % n == 0:
                        num_attention_heads = n
                        break
            
            self.self_attention = MultiHeadSelfAttention(
                hidden_dim=self.lstm_output_dim,
                num_heads=num_attention_heads,
                dropout=dropout,
                use_layer_norm=use_layer_norm
            )
        
        # --- Output Head ---
        self.dropout = nn.Dropout(dropout)
        
        # Final projection layer
        if use_layer_norm:
            self.pre_head_norm = nn.LayerNorm(self.lstm_output_dim)
        
        self.head = nn.Sequential(
            nn.Linear(self.lstm_output_dim, hidden_dim),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass.
        
        Args:
            x: (Batch, Seq_Len, Features) input tensor
            
        Returns:
            prediction: (Batch, 1) predicted value
        """
        # 1. LSTM Encoding
        # out: (Batch, Seq_Len, Hidden) or (Batch, Seq_Len, Hidden*2) if bidirectional
        out, (h_n, c_n) = self.lstm(x)
        
        # 2. Optional Layer Norm on LSTM output
        if self.use_layer_norm:
            out = self.lstm_layer_norm(out)
        
        # 3. Optional Self-Attention
        if self.use_self_attention:
            out = self.self_attention(out)  # (Batch, Seq_Len, Hidden)
        
        # 4. Take the Last Timestep: (Batch, Hidden)
        last_hidden = out[:, -1, :]
        
        # 5. Dropout + Optional Pre-Head Norm
        out = self.dropout(last_hidden)
        if self.use_layer_norm:
            out = self.pre_head_norm(out)
        
        # 6. Final Prediction: (Batch, 1)
        prediction = self.head(out)
        
        return prediction

print("LSTMWithAttention model class defined.")

LSTMWithAttention model class defined.


#### (Continued) Seq2Seq Model with Cross-Attention (Task 2)

Encoder-Decoder with stacked layers, cross-attention, and teacher forcing.

In [9]:
# ==============================================================================
# CROSS-ATTENTION MODULE
# ==============================================================================

class CrossAttention(nn.Module):
    """
    Bahdanau-style Additive Attention.
    
    Computes attention weights between decoder hidden state and encoder outputs,
    then returns weighted context vector.
    
    Note: This module handles the case where decoder hidden dim differs from 
    encoder output dim (e.g., when encoder is bidirectional).
    
    Args:
        encoder_dim: Dimension of encoder outputs (hidden*2 if bidirectional)
        decoder_dim: Dimension of decoder hidden state (optional, defaults to encoder_dim)
    """
    def __init__(self, encoder_dim: int, decoder_dim: int = None):
        super(CrossAttention, self).__init__()
        
        if decoder_dim is None:
            decoder_dim = encoder_dim
        
        self.encoder_dim = encoder_dim
        self.decoder_dim = decoder_dim
        
        # Project both to the same space for attention computation
        self.attn = nn.Linear(encoder_dim + decoder_dim, encoder_dim)
        self.v = nn.Linear(encoder_dim, 1, bias=False)

    def forward(self, hidden: torch.Tensor, encoder_outputs: torch.Tensor) -> tuple:
        """
        Args:
            hidden: Decoder state at current step (Batch, Decoder_Dim)
            encoder_outputs: All encoder states (Batch, Seq_Len, Encoder_Dim)
            
        Returns:
            context: Weighted sum of encoder outputs (Batch, Encoder_Dim)
            weights: Attention weights (Batch, Seq_Len)
        """
        seq_len = encoder_outputs.size(1)
        
        # Repeat decoder hidden state: (Batch, Seq_Len, Decoder_Dim)
        hidden_expanded = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        
        # Calculate Energy: score = v * tanh(W * [h_dec; h_enc])
        combined = torch.cat((hidden_expanded, encoder_outputs), dim=2)  # (Batch, Seq, Dec+Enc)
        energy = torch.tanh(self.attn(combined))  # (Batch, Seq, Encoder_Dim)
        attention = self.v(energy).squeeze(2)  # (Batch, Seq_Len)
        
        # Softmax to get weights
        weights = F.softmax(attention, dim=1)
        
        # Compute Context Vector
        # (Batch, 1, Seq_Len) @ (Batch, Seq_Len, Encoder_Dim) -> (Batch, 1, Encoder_Dim)
        context = torch.bmm(weights.unsqueeze(1), encoder_outputs)
        
        return context.squeeze(1), weights

print("CrossAttention module defined.")

CrossAttention module defined.


In [10]:
# ==============================================================================
# ENHANCED LSTM SEQ2SEQ MODEL WITH ATTENTION (Task 2: Multi-Step Prediction)
# ==============================================================================

class LSTM_Seq2Seq(nn.Module):
    """
    Enhanced Encoder-Decoder with Cross-Attention for multi-step prediction.
    
    Architecture Options:
        - Bidirectional Encoder: Captures forward and backward context
        - Stacked Encoder Layers: Multiple LSTM layers in encoder
        - Stacked Decoder Layers: Multiple LSTMCell layers in decoder
        - Encoder Self-Attention: Multi-head attention on encoded sequence
        - Layer Normalization: Stabilizes training throughout
        - Cross-Attention: Bahdanau-style attention decoder-to-encoder
    
    Args:
        input_dim: Input features for Encoder (Past Dyn + Static)
        hidden_dim: LSTM hidden units (will be doubled if bidirectional for encoder output)
        future_forcing_dim: Number of known future features (weather)
        static_dim: Number of static attributes
        output_steps: Prediction horizon (default: 5)
        num_encoder_layers: Number of stacked LSTM layers in encoder
        num_decoder_layers: Number of stacked LSTMCell layers in decoder
        use_bidirectional_encoder: Use bidirectional LSTM encoder
        use_self_attention: Apply self-attention on encoder outputs
        num_attention_heads: Number of heads for encoder self-attention
        use_layer_norm: Apply layer normalization
        dropout: Dropout rate
    """
    def __init__(self, input_dim: int, hidden_dim: int, future_forcing_dim: int, 
                 static_dim: int, output_steps: int = 5,
                 num_encoder_layers: int = 1,
                 num_decoder_layers: int = 1,
                 use_bidirectional_encoder: bool = True,
                 use_self_attention: bool = True,
                 num_attention_heads: int = 4,
                 use_layer_norm: bool = True,
                 dropout: float = 0.2):
        super(LSTM_Seq2Seq, self).__init__()
        
        self.output_steps = output_steps
        self.static_dim = static_dim
        self.hidden_dim = hidden_dim
        self.use_bidirectional_encoder = use_bidirectional_encoder
        self.use_self_attention = use_self_attention
        self.use_layer_norm = use_layer_norm
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.dropout_rate = dropout
        
        # Encoder output dimension (doubled if bidirectional)
        self.encoder_output_dim = hidden_dim * 2 if use_bidirectional_encoder else hidden_dim
        
        # --- ENCODER (Stacked LSTM) ---
        self.encoder = nn.LSTM(
            input_dim, hidden_dim,
            num_layers=num_encoder_layers,
            batch_first=True,
            bidirectional=use_bidirectional_encoder,
            dropout=dropout if num_encoder_layers > 1 else 0
        )
        
        # Optional Layer Norm after encoder
        if use_layer_norm:
            self.encoder_layer_norm = nn.LayerNorm(self.encoder_output_dim)
        
        # Optional Self-Attention on encoder outputs
        if use_self_attention:
            # Adjust num_heads if needed
            adjusted_heads = num_attention_heads
            if self.encoder_output_dim % num_attention_heads != 0:
                for n in range(num_attention_heads, 0, -1):
                    if self.encoder_output_dim % n == 0:
                        adjusted_heads = n
                        break
            
            self.encoder_self_attention = MultiHeadSelfAttention(
                hidden_dim=self.encoder_output_dim,
                num_heads=adjusted_heads,
                dropout=dropout,
                use_layer_norm=use_layer_norm
            )
        
        # --- DECODER COMPONENTS ---
        # For multi-layer encoder, we take the last layer's hidden state
        if use_bidirectional_encoder:
            # Project concatenated forward+backward states to decoder hidden size
            self.hidden_projection = nn.Linear(hidden_dim * 2, hidden_dim)
            self.cell_projection = nn.Linear(hidden_dim * 2, hidden_dim)
        
        # Decoder Input: Previous_Flow (1) + Future_Forcing + Static
        decoder_input_dim = 1 + future_forcing_dim + static_dim
        
        # Stacked Decoder LSTMCells
        # First layer takes decoder_input_dim, subsequent layers take hidden_dim
        self.decoder_cells = nn.ModuleList()
        for layer_idx in range(num_decoder_layers):
            input_size = decoder_input_dim if layer_idx == 0 else hidden_dim
            self.decoder_cells.append(nn.LSTMCell(input_size, hidden_dim))
        
        # Dropout between decoder layers (only if > 1 layer)
        self.decoder_dropout = nn.Dropout(dropout) if num_decoder_layers > 1 else None
        
        # Cross-attention: encoder_dim = encoder_output_dim, decoder_dim = hidden_dim
        self.attention = CrossAttention(
            encoder_dim=self.encoder_output_dim,
            decoder_dim=hidden_dim
        )
        
        # Optional decoder layer norm (applied to top layer)
        if use_layer_norm:
            self.decoder_layer_norm = nn.LayerNorm(hidden_dim)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Final projection: [Decoder_Hidden + Context] -> Output
        # Context comes from encoder (encoder_output_dim), decoder hidden is hidden_dim
        self.fc_out = nn.Linear(hidden_dim + self.encoder_output_dim, 1)

    def forward(self, x_past: torch.Tensor, x_future_forcing: torch.Tensor, 
                static_features: torch.Tensor, target_seq: torch.Tensor = None, 
                teacher_forcing_ratio: float = 0.5) -> torch.Tensor:
        """
        Forward pass with optional teacher forcing.
        
        Supports stacked encoder (num_encoder_layers) and stacked decoder (num_decoder_layers).
        
        Args:
            x_past: (Batch, Past_Seq, Input_Dim) - Past sequence
            x_future_forcing: (Batch, output_steps, Future_Forcing_Dim) - Known future weather
            static_features: (Batch, Static_Dim) - Static catchment attributes
            target_seq: (Batch, output_steps) - Ground truth (for teacher forcing)
            teacher_forcing_ratio: Probability of using ground truth
            
        Returns:
            outputs: (Batch, output_steps) - Predicted flow sequence
        """
        batch_size = x_past.size(0)
        
        # =====================================================================
        # 1. ENCODE
        # =====================================================================
        # encoder_outputs: (Batch, Seq, Hidden) or (Batch, Seq, Hidden*2) if bidirectional
        # enc_hidden, enc_cell: (num_layers * num_directions, Batch, Hidden)
        encoder_outputs, (enc_hidden, enc_cell) = self.encoder(x_past)
        
        # 2. Optional Layer Norm on encoder outputs
        if self.use_layer_norm:
            encoder_outputs = self.encoder_layer_norm(encoder_outputs)
        
        # 3. Optional Self-Attention on encoder outputs
        if self.use_self_attention:
            encoder_outputs = self.encoder_self_attention(encoder_outputs)
        
        # =====================================================================
        # 4. Initialize decoder hidden states from encoder's last layer
        # =====================================================================
        # For multi-layer encoder, use only the last layer's hidden states
        # For bidirectional, last layer indices are: [-2] (forward), [-1] (backward)
        if self.use_bidirectional_encoder:
            # Get last layer's forward and backward states
            # Shape of enc_hidden: (num_layers * 2, Batch, Hidden)
            h_forward = enc_hidden[-2]  # (Batch, Hidden)
            h_backward = enc_hidden[-1]  # (Batch, Hidden)
            c_forward = enc_cell[-2]
            c_backward = enc_cell[-1]
            
            # Concatenate and project to decoder hidden size
            hidden_cat = torch.cat((h_forward, h_backward), dim=1)  # (Batch, Hidden*2)
            cell_cat = torch.cat((c_forward, c_backward), dim=1)    # (Batch, Hidden*2)
            
            init_hidden = self.hidden_projection(hidden_cat)  # (Batch, Hidden)
            init_cell = self.cell_projection(cell_cat)        # (Batch, Hidden)
        else:
            # For unidirectional, take last layer: enc_hidden[-1]
            init_hidden = enc_hidden[-1]  # (Batch, Hidden)
            init_cell = enc_cell[-1]      # (Batch, Hidden)
        
        # Initialize hidden states for each decoder layer
        # All layers start with the same initial state (from encoder)
        decoder_hiddens = [init_hidden.clone() for _ in range(self.num_decoder_layers)]
        decoder_cells = [init_cell.clone() for _ in range(self.num_decoder_layers)]
        
        # Initialize outputs container
        outputs = torch.zeros(batch_size, self.output_steps).to(x_past.device)
        
        # First decoder input is the Last Observed Flow (last column of x_past)
        decoder_input_flow = x_past[:, -1, -1].unsqueeze(1)  # (Batch, 1)
        
        # =====================================================================
        # 5. DECODE LOOP (t=0 to output_steps-1)
        # =====================================================================
        for t in range(self.output_steps):
            # --- Prepare Decoder Input ---
            # Structure: [Flow(t-1), Future_Forcing(t), Static]
            current_forcing = x_future_forcing[:, t, :]  # (Batch, Forcing_Dim)
            
            inputs_list = [decoder_input_flow, current_forcing]
            if self.static_dim > 0 and static_features is not None:
                inputs_list.append(static_features)
            
            dec_input = torch.cat(inputs_list, dim=1)  # (Batch, decoder_input_dim)
            
            # --- Run Stacked Decoder Cells ---
            layer_input = dec_input
            for layer_idx in range(self.num_decoder_layers):
                decoder_hiddens[layer_idx], decoder_cells[layer_idx] = self.decoder_cells[layer_idx](
                    layer_input, (decoder_hiddens[layer_idx], decoder_cells[layer_idx])
                )
                # Apply dropout between layers (not after the last layer)
                if self.decoder_dropout is not None and layer_idx < self.num_decoder_layers - 1:
                    layer_input = self.decoder_dropout(decoder_hiddens[layer_idx])
                else:
                    layer_input = decoder_hiddens[layer_idx]
            
            # Use the top layer's hidden state for attention and prediction
            top_hidden = decoder_hiddens[-1]
            
            # Optional layer norm on decoder hidden
            if self.use_layer_norm:
                hidden_normed = self.decoder_layer_norm(top_hidden)
            else:
                hidden_normed = top_hidden
            
            # --- Cross Attention ---
            context, _ = self.attention(hidden_normed, encoder_outputs)
            
            # --- Dropout ---
            hidden_dropped = self.dropout(hidden_normed)
            context_dropped = self.dropout(context)
            
            # --- Predict ---
            combined = torch.cat((hidden_dropped, context_dropped), dim=1)
            prediction = self.fc_out(combined)  # (Batch, 1)
            
            # Store prediction
            outputs[:, t] = prediction.squeeze(1)
            
            # --- Teacher Forcing Logic ---
            if target_seq is not None and torch.rand(1).item() < teacher_forcing_ratio:
                decoder_input_flow = target_seq[:, t].unsqueeze(1)
            else:
                decoder_input_flow = prediction
        
        return outputs

print("LSTM_Seq2Seq model class (enhanced with stacked layers) defined.")

LSTM_Seq2Seq model class (enhanced with stacked layers) defined.


### 1.4 Utility Functions (Metrics & Weight Init)

Metrics calculation (NSE, RMSE, etc.) and weight initialization utilities.

In [11]:
# ==============================================================================
# UTILITY FUNCTIONS
# ==============================================================================

def calc_nse(obs: np.ndarray, sim: np.ndarray) -> float:
    """
    Nash-Sutcliffe Efficiency (NSE).
    
    NSE = 1 - (sum((obs - sim)^2) / sum((obs - mean(obs))^2))
    
    Args:
        obs: Observed values
        sim: Simulated/predicted values
        
    Returns:
        NSE value. Perfect prediction = 1.0, mean prediction = 0.0
    """
    denominator = np.sum((obs - np.mean(obs)) ** 2) + 1e-6
    numerator = np.sum((sim - obs) ** 2)
    return 1 - (numerator / denominator)


def calc_metrics(obs: np.ndarray, sim: np.ndarray) -> Dict[str, float]:
    """Calculate comprehensive evaluation metrics."""
    mse = np.mean((obs - sim) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(obs - sim))
    nse = calc_nse(obs, sim)
    
    # R-squared
    ss_res = np.sum((obs - sim) ** 2)
    ss_tot = np.sum((obs - np.mean(obs)) ** 2) + 1e-6
    r2 = 1 - (ss_res / ss_tot)
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'NSE': nse,
        'R2': r2
    }

print("Utility functions defined.")

Utility functions defined.


#### (Continued) Weight Initialization

Xavier/Kaiming initialization for LSTM and linear layers.

In [12]:
# ==============================================================================
# WEIGHT INITIALIZATION UTILITIES
# ==============================================================================

def init_weights(module: nn.Module) -> None:
    """
    Apply proper weight initialization to a single module.
    
    Initialization Strategy:
        - Linear layers: Xavier uniform (good for tanh/sigmoid activations)
        - LSTM layers: Orthogonal for weights, zeros for biases
        - LayerNorm: weights=1, bias=0
    
    Args:
        module: A single nn.Module instance
    """
    if isinstance(module, nn.Linear):
        # Xavier initialization for linear layers
        nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            nn.init.zeros_(module.bias)
    
    elif isinstance(module, (nn.LSTM, nn.LSTMCell)):
        # Orthogonal initialization for LSTM weights (prevents vanishing/exploding gradients)
        for name, param in module.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                nn.init.zeros_(param.data)
                # Set forget gate bias to 1 for better gradient flow
                # LSTM has 4 gates: [input, forget, cell, output]
                n = param.size(0)
                param.data[n // 4: n // 2].fill_(1.0)
    
    elif isinstance(module, nn.LayerNorm):
        nn.init.ones_(module.weight)
        nn.init.zeros_(module.bias)


def apply_weight_init(model: nn.Module) -> nn.Module:
    """
    Apply weight initialization to all modules in a model.
    
    Args:
        model: The full model to initialize
        
    Returns:
        The initialized model (for chaining)
    """
    model.apply(init_weights)
    return model

print("Weight initialization utilities defined.")

Weight initialization utilities defined.


### 1.5 Dataset Preparation

Data loading and dataset generation functions for training/validation/test splits.

In [13]:
# ==============================================================================
# DATA LOADING & PREPARATION FUNCTION
# ==============================================================================

def load_and_prepare_data(cfg: Union[Task1Config, Task2Config]) -> Tuple[Dict[str, pd.DataFrame], pd.DataFrame, 
                                                                          CamelsPreprocessor, List[str]]:
    """
    Complete data loading and preparation pipeline.
    
    This function orchestrates:
    1. Loading basin list (excluding bad basins)
    2. Selecting basins based on cfg.num_basins
    3. Loading static attributes
    4. Loading and processing dynamic data for all basins
    5. Fitting the preprocessor on training data
    
    Args:
        cfg: Task1Config or Task2Config with all settings
        
    Returns:
        dynamic_data: {gauge_id: DataFrame} of processed dynamic data
        df_static: DataFrame of static features (or None if use_static=False)
        preprocessor: Fitted CamelsPreprocessor
        basin_ids: List of selected basin gauge IDs
    """
    print("=" * 60)
    print("LOADING AND PREPARING DATA")
    print("=" * 60)
    
    # 1. Initialize loader and get basin list
    loader = CamelsLoader(cfg)
    df_basins = loader.get_basin_list()
    print(f"\nTotal valid basins found: {len(df_basins)}")
    
    # 2. Select basins based on num_basins parameter
    if cfg.num_basins > 0:
        print(f"Selecting first {cfg.num_basins} basins for experiment...")
        df_basins = df_basins.head(cfg.num_basins)
    else:
        print(f"Using ALL {len(df_basins)} basins for full training...")
    
    basin_ids = df_basins['gauge_id'].tolist()
    print(f"Selected basins: {basin_ids[:5]}{'...' if len(basin_ids) > 5 else ''}")
    
    # 3. Load static attributes
    if cfg.use_static:
        df_static = loader.load_static_attributes(basin_ids)
        print(f"\nStatic attributes loaded: {df_static.shape}")
    else:
        df_static = None
        print("\nStatic features disabled (use_static=False)")
    
    # 4. Initialize preprocessor and feature engineer
    preprocessor = CamelsPreprocessor(cfg)
    engineer = FeatureEngineer(cfg)
    
    # 5. Load and process all basins
    dynamic_data = {}
    print(f"\nLoading and preprocessing {len(df_basins)} basins...")
    print("Processing order: Clean Outliers → Feature Engineering → Interpolation → Date Features")
    
    for _, row in tqdm(df_basins.iterrows(), total=len(df_basins), desc="Loading basins"):
        gid = row['gauge_id']
        region = row['region']
        
        df = loader.load_dynamic_data(gid, region)
        if df is not None:
            # Order matters to avoid data leakage:
            df = preprocessor.clean_physical_outliers(df)
            df = engineer.transform(df)
            df = preprocessor.handle_missing_data(df)
            df = preprocessor.add_date_features(df)
            dynamic_data[gid] = df
    
    print(f"\nSuccessfully loaded {len(dynamic_data)} basins")
    
    # 6. Fit preprocessor on training data
    print("\n" + "=" * 60)
    print("FITTING PREPROCESSOR")
    print("=" * 60)
    preprocessor.fit(dynamic_data, df_static)
    
    return dynamic_data, df_static, preprocessor, basin_ids

print("load_and_prepare_data function defined.")

load_and_prepare_data function defined.


#### (Continued) Dataset Generation Functions

Sequence creation for Task 1 (single-step) and Task 2 (multi-step with future forcing).

In [14]:
# ==============================================================================
# DATASET GENERATION FUNCTIONS
# ==============================================================================

def get_task1_dataset(cfg: Task1Config, dynamic_data: Dict[str, pd.DataFrame], 
                      df_static: pd.DataFrame, preprocessor: 'CamelsPreprocessor',
                      basin_ids: List[str], split: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generate dataset for Task 1 (single-step prediction).
    
    Args:
        cfg: Task1Config
        dynamic_data: {gauge_id: DataFrame} of processed data
        df_static: Static features DataFrame
        preprocessor: Fitted preprocessor
        basin_ids: List of basin IDs
        split: 'train', 'val', or 'test'
        
    Returns:
        X: (N, Seq_Len, Features) input sequences
        y: (N,) target values
    """
    # Get date range for split
    if split == 'train':
        start, end = cfg.train_start, cfg.train_end
    elif split == 'val':
        start, end = cfg.val_start, cfg.val_end
    else:
        start, end = cfg.test_start, cfg.test_end
    
    X_list, y_list = [], []
    
    for gid in basin_ids:
        if gid not in dynamic_data:
            continue
            
        df = dynamic_data[gid]
        df_slice = df.loc[start:end]
        
        if len(df_slice) < cfg.seq_length + cfg.predict_horizon:
            continue
        
        # Transform and create sequences
        data_matrix, static_norm = preprocessor.transform(df_slice, df_static, gid)
        
        if cfg.use_static and static_norm is not None:
            X_seq, y_seq = preprocessor.create_sequences(data_matrix, static_norm, mode='task1')
        else:
            X_seq, y_seq = preprocessor.create_sequences(data_matrix, None, mode='task1')
        
        if len(X_seq) > 0:
            X_list.append(X_seq)
            y_list.append(y_seq)
    
    if not X_list:
        return np.array([]), np.array([])
    
    return np.concatenate(X_list), np.concatenate(y_list)


def get_task2_dataset(cfg: Task2Config, dynamic_data: Dict[str, pd.DataFrame], 
                      df_static: pd.DataFrame, preprocessor: 'CamelsPreprocessor',
                      basin_ids: List[str], split: str) -> Tuple[np.ndarray, np.ndarray, 
                                                                   np.ndarray, np.ndarray]:
    """
    Generate dataset for Task 2 (multi-step prediction with future forcing).
    
    Args:
        cfg: Task2Config
        dynamic_data: {gauge_id: DataFrame} of processed data
        df_static: Static features DataFrame
        preprocessor: Fitted preprocessor
        basin_ids: List of basin IDs
        split: 'train', 'val', or 'test'
        
    Returns:
        X_past: (N, Seq_Len, Features) past sequences
        X_future: (N, 5, Future_Forcing_Dim) future forcing
        Static: (N, Static_Dim) static features
        y: (N, 5) target sequences
    """
    # Get date range for split
    if split == 'train':
        start, end = cfg.train_start, cfg.train_end
    elif split == 'val':
        start, end = cfg.val_start, cfg.val_end
    else:
        start, end = cfg.test_start, cfg.test_end
    
    X_past_list, X_future_list, Static_list, Y_list = [], [], [], []
    
    # Future forcing features (known weather forecasts)
    future_forcing_cols = cfg.future_forcing_features
    
    for gid in basin_ids:
        if gid not in dynamic_data:
            continue
            
        df = dynamic_data[gid]
        df_slice = df.loc[start:end]
        
        if len(df_slice) < cfg.seq_length + cfg.predict_steps:
            continue
        
        # Transform
        data_matrix, static_norm = preprocessor.transform(df_slice, df_static, gid)
        
        # Create sequences
        if cfg.use_static and static_norm is not None:
            X_seq, y_seq = preprocessor.create_sequences(data_matrix, static_norm, mode='task2')
        else:
            X_seq, y_seq = preprocessor.create_sequences(data_matrix, None, mode='task2')
        
        if len(X_seq) == 0:
            continue
        
        # Extract future forcing (from the slice, not normalized data)
        # Need to align with the sequences
        n_samples = len(X_seq)
        future_forcing = []
        
        # Get normalized future forcing
        dyn_cols = [c for c in cfg.dynamic_features if c in df_slice.columns]
        forcing_indices = [dyn_cols.index(c) for c in future_forcing_cols if c in dyn_cols]
        
        for i in range(n_samples):
            # Sequence starts at index i, ends at i + seq_length
            # Future is from i + seq_length to i + seq_length + predict_steps
            future_start = cfg.seq_length + i
            future_end = future_start + cfg.predict_steps
            
            if future_end <= len(data_matrix):
                future_feats = data_matrix[future_start:future_end, forcing_indices]
                future_forcing.append(future_feats)
        
        if len(future_forcing) != n_samples:
            continue
        
        X_past_list.append(X_seq)
        X_future_list.append(np.array(future_forcing))
        Y_list.append(y_seq)
        
        # Static features for each sample
        if static_norm is not None:
            Static_list.append(np.tile(static_norm, (n_samples, 1)))
        else:
            Static_list.append(np.zeros((n_samples, len(cfg.static_features))))
    
    if not X_past_list:
        return np.array([]), np.array([]), np.array([]), np.array([])
    
    return (np.concatenate(X_past_list), np.concatenate(X_future_list),
            np.concatenate(Static_list), np.concatenate(Y_list))

print("Dataset generation functions defined.")

Dataset generation functions defined.


### 1.6 Training & Evaluation

Training loops with early stopping, scheduler, gradient clipping. Evaluation with metrics and visualization.

In [15]:
# ==============================================================================
# TRAINING FUNCTION FOR TASK 1 (with Toggleable DL Techniques)
# ==============================================================================

def train_task1(cfg: Task1Config, model: nn.Module, 
                train_loader: DataLoader, val_loader: DataLoader) -> Tuple[nn.Module, Dict[str, List[float]]]:
    """
    Train Task 1 LSTM model with early stopping and toggleable advanced techniques.
    
    Advanced Techniques (controlled by cfg):
    - Weight Decay via AdamW (cfg.use_weight_decay)
    - Learning Rate Scheduler (cfg.use_scheduler)
    - Gradient Clipping (cfg.use_gradient_clipping)
    
    Args:
        cfg: Task1Config with training settings
        model: LSTM model to train
        train_loader: Training data loader
        val_loader: Validation data loader
        
    Returns:
        model: Trained model (with best weights loaded)
        history: {'train_loss': [...], 'val_loss': [...]}
    """
    print("=" * 60)
    print("TRAINING TASK 1: Single-Step LSTM")
    print("=" * 60)
    print(f"Epochs: {cfg.epochs}, Patience: {cfg.patience}, LR: {cfg.learning_rate}")
    print(f"Weight Decay: {cfg.use_weight_decay} (λ={cfg.weight_decay})")
    print(f"LR Scheduler: {cfg.use_scheduler}")
    print(f"Gradient Clipping: {cfg.use_gradient_clipping} (max_norm={cfg.gradient_clip_value})")
    
    # === Optimizer Selection ===
    if cfg.use_weight_decay:
        optimizer = optim.AdamW(model.parameters(), 
                                lr=cfg.learning_rate, 
                                weight_decay=cfg.weight_decay)
    else:
        optimizer = optim.Adam(model.parameters(), lr=cfg.learning_rate)
    
    # === Learning Rate Scheduler (optional) ===
    scheduler = None
    if cfg.use_scheduler:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', 
            factor=cfg.scheduler_factor, 
            patience=cfg.scheduler_patience
        )

    criterion = nn.MSELoss()
    
    history = {'train_loss': [], 'val_loss': []}
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    
    for epoch in range(cfg.epochs):
        # --- Training ---
        model.train()
        train_loss = 0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(cfg.device), y_batch.to(cfg.device)
            
            optimizer.zero_grad()
            predictions = model(X_batch).squeeze()
            loss = criterion(predictions, y_batch)
            loss.backward()
            
            # === Gradient Clipping (optional) ===
            if cfg.use_gradient_clipping:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=cfg.gradient_clip_value)
            
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        history['train_loss'].append(train_loss)
        
        # --- Validation ---
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(cfg.device), y_batch.to(cfg.device)
                predictions = model(X_batch).squeeze()
                val_loss += criterion(predictions, y_batch).item()
        
        val_loss /= len(val_loader)
        history['val_loss'].append(val_loss)

        # === Step Scheduler (after validation) ===
        current_lr = optimizer.param_groups[0]['lr']
        if scheduler is not None:
            scheduler.step(val_loss)
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = copy.deepcopy(model.state_dict())
            patience_counter = 0
            save_marker = " *** Best ***"
        else:
            patience_counter += 1
            save_marker = f" (patience: {patience_counter}/{cfg.patience})"
        
        print(f"Epoch {epoch+1:02d}/{cfg.epochs}: Train = {train_loss:.6f} | Val = {val_loss:.6f} | LR = {current_lr:.1e}{save_marker}")
        
        # Early stopping
        if patience_counter >= cfg.patience:
            print(f"\nEarly stopping triggered at epoch {epoch+1}")
            break
    
    print("=" * 60)
    print(f"Training complete. Best Val Loss: {best_val_loss:.6f}")
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, history


# ==============================================================================
# TRAINING FUNCTION FOR TASK 2 (with Toggleable DL Techniques)
# ==============================================================================

def train_task2(cfg: Task2Config, model: nn.Module,
                train_loader: DataLoader, val_loader: DataLoader) -> Tuple[nn.Module, Dict[str, List[float]]]:
    """
    Train Task 2 Seq2Seq model with early stopping, teacher forcing decay, 
    and toggleable advanced techniques.
    
    Advanced Techniques (controlled by cfg):
    - Weight Decay via AdamW (cfg.use_weight_decay)
    - Learning Rate Scheduler (cfg.use_scheduler)
    - Gradient Clipping (cfg.use_gradient_clipping)
    
    Args:
        cfg: Task2Config with training settings
        model: LSTM_Seq2Seq model to train
        train_loader: Training data loader
        val_loader: Validation data loader
        
    Returns:
        model: Trained model (with best weights loaded)
        history: {'train_loss': [...], 'val_loss': [...]}
    """
    print("=" * 60)
    print("TRAINING TASK 2: Multi-Step Seq2Seq with Attention")
    print("=" * 60)
    print(f"Epochs: {cfg.epochs}, Patience: {cfg.patience}, LR: {cfg.learning_rate}")
    print(f"Teacher Forcing: {cfg.teacher_forcing_ratio}, Decay: {cfg.tf_decay}")
    print(f"Weight Decay: {cfg.use_weight_decay} (λ={cfg.weight_decay})")
    print(f"LR Scheduler: {cfg.use_scheduler}")
    print(f"Gradient Clipping: {cfg.use_gradient_clipping} (max_norm={cfg.gradient_clip_value})")
    
    # === Optimizer Selection ===
    if cfg.use_weight_decay:
        optimizer = optim.AdamW(model.parameters(), 
                                lr=cfg.learning_rate, 
                                weight_decay=cfg.weight_decay)
    else:
        optimizer = optim.Adam(model.parameters(), lr=cfg.learning_rate)
    
    # === Learning Rate Scheduler (optional) ===
    scheduler = None
    if cfg.use_scheduler:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', 
            factor=cfg.scheduler_factor, 
            patience=cfg.scheduler_patience
        )
    
    criterion = nn.MSELoss()
    
    history = {'train_loss': [], 'val_loss': []}
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    
    for epoch in range(cfg.epochs):
        # Teacher forcing ratio (optionally decay)
        if cfg.tf_decay:
            tf_ratio = cfg.teacher_forcing_ratio * (1 - epoch / cfg.epochs)
        else:
            tf_ratio = cfg.teacher_forcing_ratio
        
        # --- Training ---
        model.train()
        train_loss = 0
        
        for x_past, x_future, static, y in train_loader:
            x_past = x_past.to(cfg.device)
            x_future = x_future.to(cfg.device)
            static = static.to(cfg.device)
            y = y.to(cfg.device)
            
            optimizer.zero_grad()
            predictions = model(x_past, x_future, static, target_seq=y, teacher_forcing_ratio=tf_ratio)
            loss = criterion(predictions, y)
            loss.backward()
            
            # === Gradient Clipping (optional) ===
            if cfg.use_gradient_clipping:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=cfg.gradient_clip_value)
            
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        history['train_loss'].append(train_loss)
        
        # --- Validation (no teacher forcing) ---
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for x_past, x_future, static, y in val_loader:
                x_past = x_past.to(cfg.device)
                x_future = x_future.to(cfg.device)
                static = static.to(cfg.device)
                y = y.to(cfg.device)
                
                predictions = model(x_past, x_future, static, target_seq=None, teacher_forcing_ratio=0)
                val_loss += criterion(predictions, y).item()
        
        val_loss /= len(val_loader)
        history['val_loss'].append(val_loss)
        
        # === Step Scheduler (after validation) ===
        current_lr = optimizer.param_groups[0]['lr']
        if scheduler is not None:
            scheduler.step(val_loss)
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = copy.deepcopy(model.state_dict())
            patience_counter = 0
            save_marker = " *** Best ***"
        else:
            patience_counter += 1
            save_marker = f" (patience: {patience_counter}/{cfg.patience})"
        
        print(f"Epoch {epoch+1:02d}/{cfg.epochs}: Train = {train_loss:.6f} | Val = {val_loss:.6f} | TF = {tf_ratio:.2f} | LR = {current_lr:.1e}{save_marker}")
        
        # Early stopping
        if patience_counter >= cfg.patience:
            print(f"\nEarly stopping triggered at epoch {epoch+1}")
            break
    
    print("=" * 60)
    print(f"Training complete. Best Val Loss: {best_val_loss:.6f}")
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, history

print("Training functions with toggleable DL techniques defined.")

Training functions with toggleable DL techniques defined.


#### (Continued) Evaluation and Visualization

Model evaluation on test set with metrics per step and visualization plots.

In [16]:
# ==============================================================================
# EVALUATION FUNCTIONS
# ==============================================================================

def evaluate_task1(cfg: Task1Config, model: nn.Module, 
                   test_loader: DataLoader) -> Tuple[np.ndarray, np.ndarray, Dict[str, float]]:
    """
    Evaluate Task 1 model on test set.
    
    Args:
        cfg: Task1Config
        model: Trained LSTM model
        test_loader: Test data loader
        
    Returns:
        predictions: (N,) predicted values
        targets: (N,) ground truth values
        metrics: Dictionary of evaluation metrics
    """
    model.eval()
    all_preds, all_targets = [], []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(cfg.device)
            predictions = model(X_batch).squeeze().cpu().numpy()
            all_preds.append(predictions)
            all_targets.append(y_batch.numpy())
    
    preds = np.concatenate(all_preds)
    targets = np.concatenate(all_targets)
    metrics = calc_metrics(targets, preds)
    
    return preds, targets, metrics


def evaluate_task2(cfg: Task2Config, model: nn.Module,
                   test_loader: DataLoader) -> Tuple[np.ndarray, np.ndarray, Dict[str, float], Dict[int, Dict[str, float]]]:
    """
    Evaluate Task 2 model on test set.
    
    Args:
        cfg: Task2Config
        model: Trained Seq2Seq model
        test_loader: Test data loader
        
    Returns:
        predictions: (N, 5) predicted sequences
        targets: (N, 5) ground truth sequences
        metrics: Dictionary of averaged metrics
        metrics_per_step: {1: {...}, 2: {...}, ...} metrics per horizon
    """
    model.eval()
    all_preds, all_targets = [], []
    
    with torch.no_grad():
        for x_past, x_future, static, y in test_loader:
            x_past = x_past.to(cfg.device)
            x_future = x_future.to(cfg.device)
            static = static.to(cfg.device)
            
            predictions = model(x_past, x_future, static, target_seq=None, teacher_forcing_ratio=0)
            all_preds.append(predictions.cpu().numpy())
            all_targets.append(y.numpy())
    
    preds = np.concatenate(all_preds)
    targets = np.concatenate(all_targets)
    
    # Calculate metrics per forecast horizon
    metrics_per_step = {}
    for step in range(cfg.predict_steps):
        metrics_per_step[step + 1] = calc_metrics(targets[:, step], preds[:, step])
    
    # Average metrics across all steps
    avg_metrics = {
        'MSE': np.mean([m['MSE'] for m in metrics_per_step.values()]),
        'RMSE': np.mean([m['RMSE'] for m in metrics_per_step.values()]),
        'MAE': np.mean([m['MAE'] for m in metrics_per_step.values()]),
        'NSE': np.mean([m['NSE'] for m in metrics_per_step.values()]),
        'R2': np.mean([m['R2'] for m in metrics_per_step.values()])
    }
    
    return preds, targets, avg_metrics, metrics_per_step


# ==============================================================================
# VISUALIZATION FUNCTIONS
# ==============================================================================

def plot_training_curves(history: Dict[str, List[float]], task_name: str, 
                         show_plots: bool = True) -> None:
    """Plot training and validation loss curves."""
    if not show_plots:
        return
    
    fig, ax = plt.subplots(figsize=(10, 5))
    
    ax.plot(history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    ax.plot(history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    best_val = min(history['val_loss'])
    ax.axhline(y=best_val, color='green', linestyle='--', alpha=0.5, label=f'Best Val: {best_val:.4f}')
    
    ax.set_xlabel('Epoch')
    ax.set_ylabel('MSE Loss')
    ax.set_title(f'{task_name}: Training & Validation Loss')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def plot_task1_results(targets: np.ndarray, preds: np.ndarray, 
                       metrics: Dict[str, float], show_plots: bool = True) -> None:
    """Plot Task 1 evaluation results."""
    if not show_plots:
        return
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Scatter plot
    ax = axes[0]
    ax.scatter(targets, preds, alpha=0.3, s=10)
    min_val, max_val = min(targets.min(), preds.min()), max(targets.max(), preds.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='1:1 Line')
    ax.set_xlabel('Observed (Normalized)')
    ax.set_ylabel('Predicted (Normalized)')
    ax.set_title(f"Task 1: Observed vs Predicted\nNSE={metrics['NSE']:.4f}, R²={metrics['R2']:.4f}")
    ax.legend()
    
    # Residuals histogram
    ax = axes[1]
    residuals = preds - targets
    ax.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(x=0, color='red', linestyle='--')
    ax.set_xlabel('Residual (Pred - Obs)')
    ax.set_ylabel('Frequency')
    ax.set_title(f'Residual Distribution\nMean={np.mean(residuals):.4f}, Std={np.std(residuals):.4f}')
    
    # Time series sample
    ax = axes[2]
    n_show = min(500, len(targets))
    ax.plot(range(n_show), targets[:n_show], 'b-', alpha=0.7, label='Observed', linewidth=1)
    ax.plot(range(n_show), preds[:n_show], 'r-', alpha=0.7, label='Predicted', linewidth=1)
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Normalized Streamflow')
    ax.set_title('Time Series Sample (First 500)')
    ax.legend()
    
    plt.tight_layout()
    plt.show()


def plot_task2_results(targets: np.ndarray, preds: np.ndarray,
                       metrics_per_step: Dict[int, Dict[str, float]], 
                       show_plots: bool = True) -> None:
    """Plot Task 2 evaluation results."""
    if not show_plots:
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # Per-step metrics
    ax = axes[0, 0]
    steps = list(metrics_per_step.keys())
    nse_values = [metrics_per_step[s]['NSE'] for s in steps]
    rmse_values = [metrics_per_step[s]['RMSE'] for s in steps]
    
    x = np.arange(len(steps))
    width = 0.35
    ax.bar(x - width/2, nse_values, width, label='NSE', color='steelblue')
    ax.set_xlabel('Forecast Horizon (t+k)')
    ax.set_ylabel('NSE')
    ax.set_xticks(x)
    ax.set_xticklabels([f't+{s}' for s in steps])
    ax.set_title('NSE by Forecast Horizon')
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    
    ax = axes[0, 1]
    ax.bar(steps, rmse_values, color='coral')
    ax.set_xlabel('Forecast Horizon (t+k)')
    ax.set_ylabel('RMSE')
    ax.set_title('RMSE by Forecast Horizon')
    
    # Scatter plot (all steps combined)
    ax = axes[0, 2]
    ax.scatter(targets.flatten(), preds.flatten(), alpha=0.2, s=5)
    min_val, max_val = targets.min(), targets.max()
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='1:1 Line')
    ax.set_xlabel('Observed (Normalized)')
    ax.set_ylabel('Predicted (Normalized)')
    ax.set_title('All Steps: Observed vs Predicted')
    
    # Sample forecast visualization
    for i, step in enumerate([1, 3, 5]):
        if step <= len(steps):
            ax = axes[1, i]
            idx = step - 1
            n_show = min(300, len(targets))
            ax.plot(range(n_show), targets[:n_show, idx], 'b-', alpha=0.7, label='Observed', linewidth=1)
            ax.plot(range(n_show), preds[:n_show, idx], 'r-', alpha=0.7, label='Predicted', linewidth=1)
            ax.set_xlabel('Sample Index')
            ax.set_ylabel('Normalized Flow')
            ax.set_title(f't+{step}: NSE={metrics_per_step[step]["NSE"]:.3f}')
            ax.legend(fontsize=8)
    
    plt.tight_layout()
    plt.show()


print("Evaluation and visualization functions defined.")

Evaluation and visualization functions defined.


### 1.7 Results Saving & Pipeline Orchestrators

Functions to save results to disk. Main orchestrators for Task 1 and Task 2 pipelines.

In [17]:
# ==============================================================================
# RESULTS SAVING FUNCTIONS (Separate for Task 1 & Task 2)
# ==============================================================================

def convert_to_serializable(obj):
    """Convert numpy types to Python native types for JSON serialization."""
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    else:
        return obj


def save_task1_results(cfg: Task1Config, results: Task1Results) -> None:
    """
    Save Task 1 pipeline results to disk.
    
    Creates:
    - results/task1/models/task1_lstm.pt
    - results/task1/metrics.json
    - results/task1/predictions.npz
    - results/task1/training_history.json
    
    Args:
        cfg: Task1Config
        results: Task1Results containing all outputs
    """
    print("=" * 60)
    print("SAVING TASK 1 RESULTS")
    print("=" * 60)
    
    # Create directories
    os.makedirs(cfg.results_dir, exist_ok=True)
    models_dir = os.path.join(cfg.results_dir, 'models')
    os.makedirs(models_dir, exist_ok=True)
    
    # Save model
    torch.save(results.model.state_dict(), os.path.join(models_dir, 'task1_lstm.pt'))
    print(f"✓ Model saved to {models_dir}/task1_lstm.pt")
    
    # Save metrics with comprehensive config
    metrics_data = {
        'metrics': convert_to_serializable(results.metrics),
        'config': {
            'num_basins': len(results.basin_ids),
            'epochs': cfg.epochs,
            'batch_size': cfg.batch_size,
            'seq_length': cfg.seq_length,
            'predict_horizon': cfg.predict_horizon,
            'patience': cfg.patience,
            'dropout': cfg.dropout,
            'learning_rate': cfg.learning_rate,
            'hidden_dim': cfg.hidden_dim,
            'bidirectional': cfg.use_bidirectional,
            'self_attention': cfg.use_self_attention,
            'attention_heads': cfg.num_attention_heads,
            'layer_norm': cfg.use_layer_norm,
            'weight_decay': cfg.weight_decay if cfg.use_weight_decay else None,
            'use_scheduler': cfg.use_scheduler,
            'use_gradient_clipping': cfg.use_gradient_clipping
        }
    }
    
    with open(os.path.join(cfg.results_dir, 'metrics.json'), 'w') as f:
        json.dump(metrics_data, f, indent=2)
    print(f"✓ Metrics saved to {cfg.results_dir}/metrics.json")
    
    # Save predictions
    np.savez(os.path.join(cfg.results_dir, 'predictions.npz'),
             predictions=results.predictions,
             targets=results.targets)
    print(f"✓ Predictions saved to {cfg.results_dir}/predictions.npz")
    
    # Save training history
    history_data = convert_to_serializable(results.history)
    with open(os.path.join(cfg.results_dir, 'training_history.json'), 'w') as f:
        json.dump(history_data, f, indent=2)
    print(f"✓ Training history saved")
    
    print("=" * 60)


def save_task2_results(cfg: Task2Config, results: Task2Results) -> None:
    """
    Save Task 2 pipeline results to disk.
    
    Creates:
    - results/task2/models/task2_seq2seq.pt
    - results/task2/metrics.json
    - results/task2/predictions.npz
    - results/task2/training_history.json
    
    Args:
        cfg: Task2Config
        results: Task2Results containing all outputs
    """
    print("=" * 60)
    print("SAVING TASK 2 RESULTS")
    print("=" * 60)
    
    # Create directories
    os.makedirs(cfg.results_dir, exist_ok=True)
    models_dir = os.path.join(cfg.results_dir, 'models')
    os.makedirs(models_dir, exist_ok=True)
    
    # Save model
    torch.save(results.model.state_dict(), os.path.join(models_dir, 'task2_seq2seq.pt'))
    print(f"✓ Model saved to {models_dir}/task2_seq2seq.pt")
    
    # Save metrics with comprehensive config
    metrics_data = {
        'metrics_average': convert_to_serializable(results.metrics),
        'metrics_per_step': {str(k): convert_to_serializable(v) for k, v in results.metrics_per_step.items()},
        'config': {
            'num_basins': len(results.basin_ids),
            'epochs': cfg.epochs,
            'batch_size': cfg.batch_size,
            'seq_length': cfg.seq_length,
            'predict_steps': cfg.predict_steps,
            'patience': cfg.patience,
            'dropout': cfg.dropout,
            'learning_rate': cfg.learning_rate,
            'hidden_dim': cfg.hidden_dim,
            'bidirectional_encoder': cfg.use_bidirectional_encoder,
            'self_attention': cfg.use_self_attention,
            'attention_heads': cfg.num_attention_heads,
            'layer_norm': cfg.use_layer_norm,
            'teacher_forcing_ratio': cfg.teacher_forcing_ratio,
            'tf_decay': cfg.tf_decay,
            'weight_decay': cfg.weight_decay if cfg.use_weight_decay else None,
            'use_scheduler': cfg.use_scheduler,
            'use_gradient_clipping': cfg.use_gradient_clipping
        }
    }
    
    with open(os.path.join(cfg.results_dir, 'metrics.json'), 'w') as f:
        json.dump(metrics_data, f, indent=2)
    print(f"✓ Metrics saved to {cfg.results_dir}/metrics.json")
    
    # Save predictions
    np.savez(os.path.join(cfg.results_dir, 'predictions.npz'),
             predictions=results.predictions,
             targets=results.targets)
    print(f"✓ Predictions saved to {cfg.results_dir}/predictions.npz")
    
    # Save training history
    history_data = convert_to_serializable(results.history)
    with open(os.path.join(cfg.results_dir, 'training_history.json'), 'w') as f:
        json.dump(history_data, f, indent=2)
    print(f"✓ Training history saved")
    
    print("=" * 60)


print("save_task1_results and save_task2_results functions defined.")

save_task1_results and save_task2_results functions defined.


#### (Continued) Pipeline Orchestrators

Separate orchestrators for running Task 1 and Task 2 pipelines independently.

In [18]:
# ==============================================================================
# TASK 1 PIPELINE ORCHESTRATOR
# ==============================================================================

def run_task1_pipeline(cfg: Task1Config) -> Task1Results:
    """
    Run the complete Task 1 pipeline: Single-step LSTM prediction (t+2).
    
    This is the main entry point for Task 1 that orchestrates:
    1. Data loading and preparation
    2. Dataset generation (train/val/test)
    3. Model building (with weight initialization)
    4. Training (with early stopping and DL techniques)
    5. Evaluation and visualization
    6. Results saving
    
    Args:
        cfg: Task1Config with all settings
        
    Returns:
        Task1Results containing model, metrics, predictions, etc.
    """
    # Print configuration
    cfg.print_config()
    
    # =========================================================================
    # STEP 1: Load and Prepare Data
    # =========================================================================
    dynamic_data, df_static, preprocessor, basin_ids = load_and_prepare_data(cfg)
    
    # =========================================================================
    # STEP 2: Generate Datasets
    # =========================================================================
    print("\n" + "=" * 60)
    print("GENERATING TASK 1 DATASETS")
    print("=" * 60)
    
    print(f"\nTask 1: Single-step prediction (t+{cfg.predict_horizon})")
    X_train, y_train = get_task1_dataset(cfg, dynamic_data, df_static, preprocessor, basin_ids, 'train')
    X_val, y_val = get_task1_dataset(cfg, dynamic_data, df_static, preprocessor, basin_ids, 'val')
    X_test, y_test = get_task1_dataset(cfg, dynamic_data, df_static, preprocessor, basin_ids, 'test')
    print(f"  Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    
    # =========================================================================
    # STEP 3: Create DataLoaders
    # =========================================================================
    train_ds = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
    val_ds = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
    test_ds = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
    
    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False)
    
    print(f"\nDataLoaders created: {len(train_loader)} training batches")
    
    # =========================================================================
    # STEP 4: Build Model (with Weight Initialization)
    # =========================================================================
    print("\n" + "=" * 60)
    print("BUILDING TASK 1 MODEL")
    print("=" * 60)
    
    input_dim = X_train.shape[2]
    model = LSTMWithAttention(
        input_dim=input_dim,
        hidden_dim=cfg.hidden_dim,
        dropout=cfg.dropout,
        num_layers=cfg.num_lstm_layers,
        use_bidirectional=cfg.use_bidirectional,
        use_self_attention=cfg.use_self_attention,
        num_attention_heads=cfg.num_attention_heads,
        use_layer_norm=cfg.use_layer_norm
    ).to(cfg.device)
    
    # Apply weight initialization
    apply_weight_init(model)
    
    print(f"\nLSTMWithAttention:")
    print(f"  Input: {input_dim}, Hidden: {cfg.hidden_dim}")
    print(f"  LSTM Layers: {cfg.num_lstm_layers}")
    print(f"  Bidirectional: {cfg.use_bidirectional}, Self-Attention: {cfg.use_self_attention}")
    print(f"  LayerNorm: {cfg.use_layer_norm}")
    print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # =========================================================================
    # STEP 5: Train Model
    # =========================================================================
    print("\n")
    model, history = train_task1(cfg, model, train_loader, val_loader)
    plot_training_curves(history, "Task 1", cfg.show_plots)
    
    # =========================================================================
    # STEP 6: Evaluate Model
    # =========================================================================
    print("\n" + "=" * 60)
    print("EVALUATING TASK 1 MODEL")
    print("=" * 60)
    
    preds, targets, metrics = evaluate_task1(cfg, model, test_loader)
    print("\nTask 1 Test Metrics:")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    plot_task1_results(targets, preds, metrics, cfg.show_plots)
    
    # =========================================================================
    # STEP 7: Create Results Object
    # =========================================================================
    results = Task1Results(
        model=model,
        history=history,
        metrics=metrics,
        predictions=preds,
        targets=targets,
        basin_ids=basin_ids,
        preprocessor=preprocessor
    )
    
    # =========================================================================
    # STEP 8: Save Results
    # =========================================================================
    save_task1_results(cfg, results)
    
    # Final summary
    results.summary()
    
    return results


# ==============================================================================
# TASK 2 PIPELINE ORCHESTRATOR
# ==============================================================================

def run_task2_pipeline(cfg: Task2Config) -> Task2Results:
    """
    Run the complete Task 2 pipeline: Multi-step Seq2Seq prediction (t+1 to t+5).
    
    This is the main entry point for Task 2 that orchestrates:
    1. Data loading and preparation
    2. Dataset generation (train/val/test) with future forcing
    3. Model building (with weight initialization)
    4. Training (with teacher forcing, early stopping, and DL techniques)
    5. Evaluation and visualization
    6. Results saving
    
    Args:
        cfg: Task2Config with all settings
        
    Returns:
        Task2Results containing model, metrics, predictions, etc.
    """
    # Print configuration
    cfg.print_config()
    
    # =========================================================================
    # STEP 1: Load and Prepare Data
    # =========================================================================
    dynamic_data, df_static, preprocessor, basin_ids = load_and_prepare_data(cfg)
    
    # =========================================================================
    # STEP 2: Generate Datasets
    # =========================================================================
    print("\n" + "=" * 60)
    print("GENERATING TASK 2 DATASETS")
    print("=" * 60)
    
    print(f"\nTask 2: Multi-step prediction (t+1 to t+{cfg.predict_steps})")
    X_past_train, X_future_train, static_train, y_train = get_task2_dataset(
        cfg, dynamic_data, df_static, preprocessor, basin_ids, 'train')
    X_past_val, X_future_val, static_val, y_val = get_task2_dataset(
        cfg, dynamic_data, df_static, preprocessor, basin_ids, 'val')
    X_past_test, X_future_test, static_test, y_test = get_task2_dataset(
        cfg, dynamic_data, df_static, preprocessor, basin_ids, 'test')
    print(f"  Train: X_past={X_past_train.shape}, y={y_train.shape}")
    print(f"  Val: X_past={X_past_val.shape}, Test: X_past={X_past_test.shape}")
    
    # =========================================================================
    # STEP 3: Create DataLoaders
    # =========================================================================
    train_ds = TensorDataset(
        torch.Tensor(X_past_train), torch.Tensor(X_future_train),
        torch.Tensor(static_train), torch.Tensor(y_train))
    val_ds = TensorDataset(
        torch.Tensor(X_past_val), torch.Tensor(X_future_val),
        torch.Tensor(static_val), torch.Tensor(y_val))
    test_ds = TensorDataset(
        torch.Tensor(X_past_test), torch.Tensor(X_future_test),
        torch.Tensor(static_test), torch.Tensor(y_test))
    
    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False)
    
    print(f"\nDataLoaders created: {len(train_loader)} training batches")
    
    # =========================================================================
    # STEP 4: Build Model (with Weight Initialization)
    # =========================================================================
    print("\n" + "=" * 60)
    print("BUILDING TASK 2 MODEL")
    print("=" * 60)
    
    input_dim = X_past_train.shape[2]
    future_dim = X_future_train.shape[2]
    static_dim = static_train.shape[1]
    
    model = LSTM_Seq2Seq(
        input_dim=input_dim,
        hidden_dim=cfg.hidden_dim,
        future_forcing_dim=future_dim,
        static_dim=static_dim,
        output_steps=cfg.predict_steps,
        num_encoder_layers=cfg.num_encoder_layers,
        num_decoder_layers=cfg.num_decoder_layers,
        use_bidirectional_encoder=cfg.use_bidirectional_encoder,
        use_self_attention=cfg.use_self_attention,
        num_attention_heads=cfg.num_attention_heads,
        use_layer_norm=cfg.use_layer_norm,
        dropout=cfg.dropout
    ).to(cfg.device)
    
    # Apply weight initialization
    apply_weight_init(model)
    
    print(f"\nLSTM_Seq2Seq:")
    print(f"  Input: {input_dim}, Hidden: {cfg.hidden_dim}")
    print(f"  Encoder Layers: {cfg.num_encoder_layers}, Decoder Layers: {cfg.num_decoder_layers}")
    print(f"  Future Forcing: {future_dim}, Static: {static_dim}")
    print(f"  Bidirectional Encoder: {cfg.use_bidirectional_encoder}")
    print(f"  Encoder Self-Attention: {cfg.use_self_attention}")
    print(f"  LayerNorm: {cfg.use_layer_norm}")
    print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # =========================================================================
    # STEP 5: Train Model
    # =========================================================================
    print("\n")
    model, history = train_task2(cfg, model, train_loader, val_loader)
    plot_training_curves(history, "Task 2", cfg.show_plots)
    
    # =========================================================================
    # STEP 6: Evaluate Model
    # =========================================================================
    print("\n" + "=" * 60)
    print("EVALUATING TASK 2 MODEL")
    print("=" * 60)
    
    preds, targets, metrics, metrics_per_step = evaluate_task2(cfg, model, test_loader)
    print("\nTask 2 Test Metrics (averaged):")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    plot_task2_results(targets, preds, metrics_per_step, cfg.show_plots)
    
    # =========================================================================
    # STEP 7: Create Results Object
    # =========================================================================
    results = Task2Results(
        model=model,
        history=history,
        metrics=metrics,
        metrics_per_step=metrics_per_step,
        predictions=preds,
        targets=targets,
        basin_ids=basin_ids,
        preprocessor=preprocessor
    )
    
    # =========================================================================
    # STEP 8: Save Results
    # =========================================================================
    save_task2_results(cfg, results)
    
    # Final summary
    results.summary()
    
    return results


print("run_task1_pipeline and run_task2_pipeline orchestrators defined.")

run_task1_pipeline and run_task2_pipeline orchestrators defined.


---
## Section 2A: Run Task 1 Pipeline

**Task 1: Single-step LSTM prediction (t+2)**

This driver cell runs the complete Task 1 pipeline independently. Modify `Task1Config` parameters below and execute.

Model Architecture Options:
- **Bidirectional LSTM**: Captures forward and backward temporal context
- **Multi-Head Self-Attention**: Learns global dependencies across the sequence  
- **Layer Normalization**: Stabilizes training and improves convergence

Advanced Training Techniques (Toggleable):
- **Weight Decay (AdamW)**: L2 regularization via `use_weight_decay`
- **LR Scheduler (ReduceLROnPlateau)**: Adaptive learning rate via `use_scheduler`
- **Gradient Clipping**: Prevents exploding gradients via `use_gradient_clipping`

In [19]:
# Remove any previous model from GPU memory
torch.cuda.empty_cache()

In [ ]:
# --- CONFIGURE TASK 1 EXPERIMENT ---
cfg1 = Task1Config(
    # === EXPERIMENT SETTINGS ===
    num_basins=150,              # 0 = ALL basins, >0 = limit for quick testing
    use_static=True,            # Include static catchment attributes
    
    # === TRAINING HYPERPARAMETERS ===
    epochs=30,
    batch_size=1024,
    learning_rate=0.001,
    dropout=0.2,
    patience=10,
    
    # === MODEL ARCHITECTURE ===
    hidden_dim=64,
    num_lstm_layers = 1,
    use_bidirectional=False,           # Bidirectional LSTM
    use_self_attention=False,          # Multi-head self-attention
    num_attention_heads=4,
    use_layer_norm=False,              # Layer normalization
    
    # === ADVANCED TRAINING TECHNIQUES (TOGGLEABLE) ===
    use_weight_decay=True,            # Use AdamW with weight decay
    weight_decay=0.01,                # L2 regularization strength
    use_scheduler=True,               # Use ReduceLROnPlateau
    scheduler_patience=3,
    scheduler_factor=0.95,
    use_gradient_clipping=False,       # Clip gradients
    gradient_clip_value=1.0,
    
    # === VISUALIZATION ===
    show_plots=True,
    
    # === PATHS ===
    results_dir='./results/task1'
)

# --- RUN TASK 1 PIPELINE ---
results_task1 = run_task1_pipeline(cfg1)

# ==============================================================================
# ACCESS TASK 1 RESULTS
# ==============================================================================
# results_task1.model           # Trained LSTMWithAttention model
# results_task1.metrics         # Test metrics: {'MSE', 'RMSE', 'MAE', 'NSE', 'R2'}
# results_task1.predictions     # Predictions array
# results_task1.targets         # Ground truth array
# results_task1.history         # Training history: {'train_loss', 'val_loss'}

TASK 1 CONFIGURATION

--- Experiment Settings ---
Number of Basins: 150 (limited)
Use Static Features: True

--- Training Hyperparameters ---
Epochs: 30, Batch Size: 1024
Learning Rate: 0.001, Dropout: 0.2
Early Stopping Patience: 10

--- Model Architecture ---
LSTM Layers: 1
Bidirectional LSTM: False
Self-Attention: False
Layer Normalization: False

--- Advanced Training Techniques ---
Weight Decay (AdamW): True (λ=0.01)
LR Scheduler: True (patience=3, factor=0.95)
Gradient Clipping: False

--- Device ---
Device: cuda
GPU: NVIDIA GeForce RTX 2050

--- Sequence Parameters ---
Sequence Length: 60 days
Prediction Horizon: t+2
LOADING AND PREPARING DATA

Total valid basins found: 608
Selecting first 150 basins for experiment...
Selected basins: ['01013500', '01022500', '01030500', '01031500', '01047000']...

Static attributes loaded: (150, 9)

Loading and preprocessing 150 basins...
Processing order: Clean Outliers → Feature Engineering → Interpolation → Date Features


Loading basins:   0%|          | 0/150 [00:00<?, ?it/s]


Successfully loaded 150 basins

FITTING PREPROCESSOR
Computing global statistics from training data...
Computing basin-specific target statistics...

GENERATING TASK 1 DATASETS

Task 1: Single-step prediction (t+2)
  Train: (790409, 60, 22), Val: (264900, 60, 22), Test: (538650, 60, 22)


---
## Section 2B: Run Task 2 Pipeline

**Task 2: Multi-step Seq2Seq prediction (t+1 to t+5)**

This driver cell runs the complete Task 2 pipeline independently. Modify `Task2Config` parameters below and execute.

Model Architecture Options:
- **Bidirectional Encoder**: Captures forward and backward context in encoder
- **Multi-Head Self-Attention**: Encoder self-attention for global dependencies
- **Cross-Attention**: Decoder attends to encoder hidden states
- **Layer Normalization**: Stabilizes training
- **Teacher Forcing**: With optional decay over epochs

Advanced Training Techniques (Toggleable):
- **Weight Decay (AdamW)**: L2 regularization via `use_weight_decay`
- **LR Scheduler (ReduceLROnPlateau)**: Adaptive learning rate via `use_scheduler`
- **Gradient Clipping**: Prevents exploding gradients via `use_gradient_clipping`

In [ ]:
# --- CONFIGURE TASK 2 EXPERIMENT ---
cfg2 = Task2Config(
    # === EXPERIMENT SETTINGS ===
    num_basins=10,              # 0 = ALL basins, >0 = limit for quick testing
    use_static=True,            # Include static catchment attributes
    
    # === TRAINING HYPERPARAMETERS ===
    epochs=100,
    batch_size=256,
    learning_rate=0.001,
    dropout=0.2,
    patience=15,
    
    # === MODEL ARCHITECTURE ===
    hidden_dim=64,
    use_bidirectional_encoder=True,   # Bidirectional encoder
    use_self_attention=True,          # Encoder self-attention
    num_attention_heads=4,
    use_layer_norm=True,              # Layer normalization
    num_encoder_layers = 2,
    num_decoder_layers = 2,

    
    # === TEACHER FORCING ===
    teacher_forcing_ratio=0.9,        # Initial TF probability
    tf_decay=True,                    # Decay TF over epochs
    
    # === ADVANCED TRAINING TECHNIQUES (TOGGLEABLE) ===
    use_weight_decay=True,            # Use AdamW with weight decay
    weight_decay=0.01,                # L2 regularization strength
    use_scheduler=True,               # Use ReduceLROnPlateau
    scheduler_patience=5,
    scheduler_factor=0.5,
    use_gradient_clipping=True,       # Clip gradients
    gradient_clip_value=1.0,
    
    # === VISUALIZATION ===
    show_plots=True,
    
    # === PATHS ===
    results_dir='./results/task2'
)

# --- RUN TASK 2 PIPELINE ---
# results_task2 = run_task2_pipeline(cfg2)

# ==============================================================================
# ACCESS TASK 2 RESULTS
# ==============================================================================
# results_task2.model           # Trained LSTM_Seq2Seq model
# results_task2.metrics         # Averaged test metrics across all steps
# results_task2.metrics_per_step  # Per-step metrics: {1: {...}, 2: {...}, ...}
# results_task2.predictions     # Predictions array (N, 5)
# results_task2.targets         # Ground truth array (N, 5)
# results_task2.history         # Training history: {'train_loss', 'val_loss'}

TASK 2 CONFIGURATION
TASK 2 CONFIGURATION

--- Experiment Settings ---
Number of Basins: 10 (limited)
Use Static Features: True

--- Training Hyperparameters ---
Epochs: 100, Batch Size: 256
Learning Rate: 0.001, Dropout: 0.2
Early Stopping Patience: 15
Encoder Layers: 1, Decoder Layers: 1
Bidirectional Encoder: True
Encoder Self-Attention: True (Heads: 4)
Layer Normalization: True
Teacher Forcing: 0.5 (Decay: True)

--- Advanced Training Techniques ---
Weight Decay (AdamW): True (λ=0.01)
LR Scheduler: True (patience=5, factor=0.5)
Gradient Clipping: True (max_norm=1.0)

--- Device ---
Device: cuda
GPU: NVIDIA GeForce RTX 2050

--- Sequence Parameters ---
Sequence Length: 60 days
Prediction Steps: 5 (t+1 to t+5)
LOADING AND PREPARING DATA

Total valid basins found: 608
Selecting first 10 basins for experiment...
Selected basins: ['01013500', '01022500', '01030500', '01031500', '01047000']...

Static attributes loaded: (10, 9)

Loading and preprocessing 10 basins...
Processing order: Cl

Loading basins:   0%|          | 0/10 [00:00<?, ?it/s]


Successfully loaded 10 basins

FITTING PREPROCESSOR
Computing global statistics from training data...
Computing basin-specific target statistics...

GENERATING TASK 2 DATASETS

Task 2: Multi-step prediction (t+1 to t+5)
  Train: X_past=(54140, 60, 22), y=(54140, 5)
  Val: X_past=(17630, 60, 22), Test: X_past=(35880, 60, 22)

DataLoaders created: 212 training batches

BUILDING TASK 2 MODEL

LSTM_Seq2Seq:
  Input: 22, Hidden: 64
  Encoder Layers: 1, Decoder Layers: 1
  Future Forcing: 5, Static: 9
  Bidirectional Encoder: True
  Encoder Self-Attention: True
  LayerNorm: True
  Parameters: 174,017


TRAINING TASK 2: Multi-Step Seq2Seq with Attention
Epochs: 100, Patience: 15, LR: 0.001
Teacher Forcing: 0.5, Decay: True
Weight Decay: True (λ=0.01)
LR Scheduler: True
Gradient Clipping: True (max_norm=1.0)
  Train: X_past=(54140, 60, 22), y=(54140, 5)
  Val: X_past=(17630, 60, 22), Test: X_past=(35880, 60, 22)

DataLoaders created: 212 training batches

BUILDING TASK 2 MODEL

LSTM_Seq2Seq:


KeyboardInterrupt: 